# Proposed Intervention Implementation Notebook  
**Paper Title**: Optimizing QoS fulfillment of Drone Services

**Conference Submission**: International Conference on Service-Oriented Computing, 2025

**Purpose**  
This notebook implements our **proposed interference-aware intervention approach**, designed to provision the charging needs of drones effectively and efficiently in the shared skyway network.

**Overview**

**Charging Allocation Strategy**
- Drones require recharging at intermediate nodes. Interferences may occur between drones at these shared nodes, leading to delays and increased energy consumption than initially anticipated by the drone providers.  
- This method introduces a **heuristic-driven priority order** to provision the charging n dronesurgency.  
- The heuristic aims to minimize the **congestion duration** at each node by efficiently assigning drones to charging and waiting pads.  
- If no pads are available, drones are temporarily placed in the **hovering zone**.  
- The strategy dynamically adjusts to drone arrival patterns and infrastructure availability.

**Objective Function and Constraints**  
- The **objective function** is to minimize the congestion duration of the nodes dynamically.
- The optimization follows a set of constraints defined in Section 6.2 of the paper.  

**Dynamic Propagation of Impact**  
- Interference impact is **not isolated** to a single node.  
- It is **propagated to subsequent nodes** in the drone's path, affecting its arrival time and battery status at future stops.  
- Therefore, nodes are processed **sequentially** to reflect realistic, dynamic propagation of delays.

**QoS Impact Assessment**  
- The impact of inter-drone interference is quantified by measuring deviations from the anticipated delivery performance.  
- For each drone, we compute:
  - Delay in delivery time due to additional waiting and time spent to recharge the battery due to hovering.
  - Additional energy consumed due to hovering at intermediate nodes.
- These values are used to evaluate the **QoS fulfillment** for each drone.
- In addition, we compute the average congestion duration of each node for the given number of drone flights operating in the shared skyway network.

**Instructions**  
- Run **all cells sequentially** to retrieve results for the proposed interference-aware intervention approach.  

**Dependencies**  
- Python 3.x  
- pandas  
- numpy  
- datetime  
(Ensure all required packages are installed before execution.)
@domain].


In [1]:
import pandas as pd
import time
import ast
import math
import os
from datetime import datetime, timedelta

In [2]:
input_file = 'Skyway_network_data_copy1.csv'
df = pd.read_csv(input_file)

# Collect data for Node1
node1_data = df[['Node1_ID', 'Waiting_Pads_Node1', 'Recharging_Pads_Node1']].copy()
node1_data.columns = ['NodeID', 'WaitingQueueLimit', 'RechargingQueueLimit']

# Collect data for Node2
node2_data = df[['Node2_ID', 'Waiting_Pads_Node2', 'Recharging_Pads_Node2']].copy()
node2_data.columns = ['NodeID', 'WaitingQueueLimit', 'RechargingQueueLimit']

# Combine both and remove duplicate node entries (keep first occurrence)
node_resources = pd.concat([node1_data, node2_data]).drop_duplicates(subset='NodeID', keep='first')

output_file = 'node_resource_count.csv'
node_resources.to_csv(output_file, index=False)

In [3]:
code_start = datetime.now()

In [4]:
requests_df = pd.read_csv('RequestsWithShortestPaths.csv')

# Parse Trajectory_Time column
def parse_trajectory_times(row):
    try:
        return ast.literal_eval(row['Trajectory_Time'])
    except:
        return {}

requests_df['Parsed_Times'] = requests_df.apply(parse_trajectory_times, axis=1)

# Collect info for each drone and node
log_data = []
for idx, row in requests_df.iterrows():
    drone_id = row['Drone_ID']
    request_id = row['Request_ID'].replace("Request_", "")
    times_dict = row['Parsed_Times']

    for node, time_range in times_dict.items():
        if node.startswith("Node_"):
            try:
                start_str, end_str = time_range.split('-')
                start_time = datetime.strptime(start_str, "%H:%M:%S.%f")
                end_time = datetime.strptime(end_str, "%H:%M:%S.%f")
                recharge_time = round((end_time - start_time).total_seconds() / 60, 2)  # in minutes

             # If the recharge time is exactly 1 minute, replace it with 0 because this shows the drone did not stop at this node; it skipped this node
                if recharge_time == 1.0:
                    recharge_time = 0

                log_data.append({
                    "Drone_ID": drone_id,
                    "RequestID": request_id,
                    "NodeID": node,
                    "ArrivalTime": start_time.strftime("%H:%M:%S"),
                    "RechargingTime": recharge_time
                })
            except Exception as e:
                print(f"Error processing node {node} for drone {drone_id}: {e}")

log_df = pd.DataFrame(log_data)
# Sort the DataFrame by NodeID and ArrivalTime
log_df.sort_values(by=["NodeID", "ArrivalTime"], inplace=True)
log_df.to_csv("chargingslot_requests_Full.csv", index=False)

In [5]:
df = pd.read_csv("chargingslot_requests_Full.csv") 
# Filter rows where RechargingTime > 0
filtered_df = df[df['RechargingTime'] > 0]
filtered_df.to_csv("chargingslot_requests_Full.csv", index=False)

In [6]:
paths_df = pd.read_csv("RequestsWithShortestPaths.csv")
charging_df = pd.read_csv("chargingslot_requests_Full.csv")

# Parse list from string
paths_df['Node_IDs_Visited'] = paths_df['Node_IDs_Visited'].apply(ast.literal_eval)

# Extract numeric request ID from string like 'Request_1212'
def extract_request_id(request_str):
    return int(str(request_str).split('_')[-1])

# Build mapping of (Drone_ID, RequestID) -> second visited node
second_node_map = {}
for _, row in paths_df.iterrows():
    drone_id = row['Drone_ID']
    request_id = extract_request_id(row['Request_ID'])
    visited_nodes = row['Node_IDs_Visited']
    if len(visited_nodes) >= 2:
        second_node_map[(drone_id, request_id)] = visited_nodes[1]

# Filter charging requests
filtered_charging_df = charging_df[
    charging_df.apply(
        lambda row: second_node_map.get((row['Drone_ID'], int(row['RequestID']))) == row['NodeID'],
        axis=1
    )
]

filtered_charging_df.to_csv("chargingslot_requests_2.csv", index=False)


In [7]:
df = pd.read_csv("chargingslot_requests_2.csv")

# Truncate seconds in the ArrivalTime column
def truncate_seconds(time_str):
    try:
        t = datetime.strptime(time_str, "%H:%M:%S")
        return t.replace(second=0).strftime("%H:%M:%S")
    except:
        return time_str

df['ArrivalTime'] = df['ArrivalTime'].apply(truncate_seconds)
df.to_csv("chargingslot_requests_2.csv", index=False)

In [8]:
# Constants
hover_power_watts = 159.01
battery_capacity_wh = 68.1
T_full_minutes = 80
factor = 0.2
tau = T_full_minutes * factor
alpha = 1.0  # heuristic-based weight factor

# Load CSV files
nodes_df = pd.read_csv('node_resource_count.csv')
drone_requests_df = pd.read_csv('chargingslot_requests_2.csv')
requests_df = pd.read_csv("RequestsWithShortestPaths.csv")

# Convert ArrivalTime
start_date = datetime.strptime("2024-06-20 07:00", "%Y-%m-%d %H:%M")
drone_requests_df['ArrivalTime'] = drone_requests_df['ArrivalTime'].apply(
    lambda x: start_date + timedelta(minutes=int(x.split(":")[0]) * 60 + int(x.split(":")[1])) - timedelta(hours=7)
)

# Preprocess Request Data
requests_df['RequestID'] = requests_df['Request_ID'].apply(lambda x: int(str(x).replace("Request_", "")))
requests_df['Battery_Consumption_Per_Segment (%)'] = requests_df['Battery_Consumption_Per_Segment (%)'].apply(ast.literal_eval)
requests_df['Node_IDs_Visited'] = requests_df['Node_IDs_Visited'].apply(ast.literal_eval)

# Battery lookup
battery_lookup = {}
for _, row in requests_df.iterrows():
    request_id = row['RequestID']
    nodes = row['Node_IDs_Visited']
    consumptions = row['Battery_Consumption_Per_Segment (%)']
    battery = 100
    battery_lookup[(request_id, nodes[0])] = battery
    for i in range(1, len(nodes)):
        battery -= consumptions[i - 1]
        battery_lookup[(request_id, nodes[i])] = battery

# Final SOC mapping
requests_df['Request_ID_Number'] = requests_df['Request_ID'].str.replace('Request_', '').astype(int)
final_soc_map = requests_df.set_index('Request_ID_Number')['Final_SOC'].to_dict()

# Queues
queues_adjusted = {
    node: {"RechargingQueue": [], "WaitingQueue": [], "OverflowQueue": []}
    for node in drone_requests_df['NodeID'].unique()
}

default_recharging_queue_limit = 1
default_waiting_queue_limit = 2

events = []
status_records = []
hover_log = []
recharge_extension_log = []

current_time = start_date
end_time = max(drone_requests_df['ArrivalTime']) + timedelta(minutes=80)

while current_time <= end_time or any(
    queues_adjusted[node]['RechargingQueue'] or
    queues_adjusted[node]['WaitingQueue'] or
    queues_adjusted[node]['OverflowQueue']
    for node in queues_adjusted.keys()
):
    new_arrivals = drone_requests_df[drone_requests_df['ArrivalTime'] == current_time]
    for _, request in new_arrivals.iterrows():
        node_id = request['NodeID']
        arrival_time = request['ArrivalTime']
        recharging_time = request['RechargingTime']
        request_id = int(request['RequestID'])

        if recharging_time == 0:
            continue

        if node_id in nodes_df['NodeID'].values:
            recharging_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'RechargingQueueLimit'].values[0]
            waiting_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'WaitingQueueLimit'].values[0]
        else:
            recharging_queue_limit = default_recharging_queue_limit
            waiting_queue_limit = default_waiting_queue_limit

        if (len(queues_adjusted[node_id]['RechargingQueue']) < recharging_queue_limit and
            len(queues_adjusted[node_id]['WaitingQueue']) == 0 and
            len(queues_adjusted[node_id]['OverflowQueue']) == 0):
            queues_adjusted[node_id]['RechargingQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "RechargingQueue (Direct Arrival - All Queues Empty)"))
        elif len(queues_adjusted[node_id]['WaitingQueue']) < waiting_queue_limit:
            queues_adjusted[node_id]['WaitingQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "WaitingQueue (New Arrival)"))
        else:
            queues_adjusted[node_id]['OverflowQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "OverflowQueue (New Arrival)"))

    for node_id in queues_adjusted.keys():
        if node_id in nodes_df['NodeID'].values:
            recharging_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'RechargingQueueLimit'].values[0]
            waiting_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'WaitingQueueLimit'].values[0]
        else:
            recharging_queue_limit = default_recharging_queue_limit
            waiting_queue_limit = default_waiting_queue_limit

        # Remove drones done recharging
        for req in queues_adjusted[node_id]['RechargingQueue'][:]:
            request_id, start_time, recharge = req
            if current_time >= start_time + timedelta(minutes=recharge):
                queues_adjusted[node_id]['RechargingQueue'].remove(req)
                events.append((current_time, request_id, "Released from RechargingQueue"))

        # heusristic based promotion to Recharging
        combined = []
        for request_id, arrival, recharge in queues_adjusted[node_id]['WaitingQueue']:
            wait_time = (current_time - arrival).total_seconds() / 60
            score = (wait_time ** 1.2) + alpha * recharge
            combined.append(("Waiting", request_id, arrival, recharge, recharge, score))

        for request_id, arrival, recharge in queues_adjusted[node_id]['OverflowQueue']:
            hover_time = (current_time - arrival).total_seconds() / 60
            battery_hover_pct = (hover_power_watts * hover_time / 60) / battery_capacity_wh * 100
            arrival_battery = battery_lookup.get((request_id, node_id), None)
            final_soc = final_soc_map.get(request_id, None)

            if arrival_battery is not None and final_soc is not None:
                total_pct = battery_hover_pct + arrival_battery
                if total_pct > 100:
                    Si = 0.11
                    Sf = final_soc
                else:
                    Si = 1 - (arrival_battery / 100)
                    Sf = Si + (battery_hover_pct / 100)
                if Sf >= 1 or Si >= Sf:
                    hover_recharge_time = 0
                else:
                    hover_recharge_time = -tau * math.log((1 - Sf) / (1 - Si))
                updated_recharge = recharge + hover_recharge_time
            else:
                updated_recharge = recharge

            wait_time = (current_time - arrival).total_seconds() / 60
            score = (wait_time ** 1.2) + alpha * updated_recharge
            combined.append(("Overflow", request_id, arrival, recharge, updated_recharge, score))

        combined.sort(key=lambda x: x[5])

        slots_available = recharging_queue_limit - len(queues_adjusted[node_id]['RechargingQueue'])
        promotions = combined[:slots_available]
        leftovers = combined[slots_available:]

        queues_adjusted[node_id]['WaitingQueue'] = []
        queues_adjusted[node_id]['OverflowQueue'] = []

        for origin, request_id, arrival, original_recharge, updated_recharge, score in promotions:
            if origin == "Overflow" and updated_recharge != original_recharge:
                queues_adjusted[node_id]['RechargingQueue'].append((request_id, current_time, updated_recharge))
                hover_time = (current_time - arrival).total_seconds() / 60
                battery_hover_pct = (hover_power_watts * hover_time / 60) / battery_capacity_wh * 100
                arrival_battery = battery_lookup.get((request_id, node_id), None)
                final_soc = final_soc_map.get(request_id, None)
                recharge_extension_log.append((request_id, node_id, original_recharge, updated_recharge, updated_recharge - original_recharge))
                hover_log.append((request_id, hover_time, battery_hover_pct, arrival_battery, final_soc, updated_recharge))
            else:
                queues_adjusted[node_id]['RechargingQueue'].append((request_id, current_time, original_recharge))

            events.append((current_time, request_id, f"Moved to RechargingQueue (WFS from {origin})"))

        # Promote remaining to waiting or overflow
        for origin, request_id, arrival, recharge, _, score in leftovers:
            if len(queues_adjusted[node_id]['WaitingQueue']) < waiting_queue_limit:
                queues_adjusted[node_id]['WaitingQueue'].append((request_id, arrival, recharge))
                events.append((current_time, request_id, f"Moved to WaitingQueue (Priority from {origin})"))
            else:
                queues_adjusted[node_id]['OverflowQueue'].append((request_id, arrival, recharge))
                events.append((current_time, request_id, f"Remains in OverflowQueue (After Priority Sort)"))

        recharging_status = [req[0] for req in queues_adjusted[node_id]['RechargingQueue']]
        waiting_status = [req[0] for req in queues_adjusted[node_id]['WaitingQueue']]
        overflow_status = [req[0] for req in queues_adjusted[node_id]['OverflowQueue']]
        status_records.append((current_time, node_id, recharging_status, waiting_status, overflow_status))

    current_time += timedelta(minutes=1)


status_df = pd.DataFrame(status_records, columns=["Time", "NodeID", "RechargingQueue", "WaitingQueue", "OverflowQueue"])
hover_df = pd.DataFrame(hover_log, columns=["Request_ID", "HoveringTime_Minutes", "Battery_Hover%", "BatteryLevelOnArrival", "Final_SOC", "UpdatedRechargeTime"])
recharge_df = pd.DataFrame(recharge_extension_log, columns=["Request_ID", "Node_ID", "Original_Recharge", "Updated_Recharge", "Increase"])
status_df.sort_values(by=["NodeID", "Time"], inplace=True)
status_df.to_csv("node_status_2.csv", index=False)

In [9]:
status_df = pd.read_csv('node_status_2.csv')
requests_df = pd.read_csv('chargingslot_requests_2.csv')

# Convert ArrivalTime to datetime
requests_df['ArrivalTime'] = requests_df['ArrivalTime'].apply(
    lambda x: datetime.strptime(f"2024-06-20 {x}", "%Y-%m-%d %H:%M:%S")
    if isinstance(x, str) and len(x) == 8 else pd.to_datetime(x)
)

# Create lookup from RequestID → (NodeID, ArrivalTime)
arrival_lookup = requests_df.set_index('RequestID')[['NodeID', 'ArrivalTime']].to_dict('index')

# Track first time a drone appears in the RechargingQueue
recharging_log = {}
# Track first time drone leaves OverflowQueue (appears in either Waiting or Recharging)
hover_exit_log = {}

for _, row in status_df.iterrows():
    time = pd.to_datetime(row['Time'])
    node_id = row['NodeID']
    
    try:
        recharging_q = eval(row['RechargingQueue'])
        waiting_q = eval(row['WaitingQueue'])
        overflow_q = eval(row['OverflowQueue'])
    except Exception:
        continue

    all_non_overflow = set(recharging_q + waiting_q)
    for request_id in recharging_q:
        key = (request_id, node_id)
        if key not in recharging_log:
            recharging_log[key] = time

    for request_id in all_non_overflow:
        key = (request_id, node_id)
        if key not in hover_exit_log:
            hover_exit_log[key] = time

# Prepare output
waiting_records = []
for (request_id, node_id), recharge_start_time in recharging_log.items():
    if request_id in arrival_lookup:
        entry = arrival_lookup[request_id]
        if entry['NodeID'] == node_id:
            arrival_time = entry['ArrivalTime']
            waiting_time = (recharge_start_time - arrival_time).total_seconds() / 60

            # Get hover exit time
            hover_exit_time = hover_exit_log.get((request_id, node_id), recharge_start_time)
            hovering_time = (hover_exit_time - arrival_time).total_seconds() / 60

            waiting_records.append({
                'RequestID': request_id,
                'NodeID': node_id,
                'ArrivalTime': arrival_time,
                'HoverExitTime': hover_exit_time,
                'RechargingStartTime': recharge_start_time,
                'HoveringTime_Minutes': round(hovering_time, 2),
                'WaitingTime_Minutes': round(waiting_time, 2)
            })

waiting_df = pd.DataFrame(waiting_records)
waiting_df.sort_values(by=['NodeID', 'ArrivalTime'], inplace=True)
waiting_df.to_csv('waiting_times_node2.csv', index=False)

In [10]:
file_path = "RequestsWithShortestPaths.csv"
df = pd.read_csv(file_path)

# Check and create Updated_Trajectory_Time column
if 'Updated_Trajectory_Time' not in df.columns:
    df['Updated_Trajectory_Time'] = df['Trajectory_Time']
else:
    print("'Updated_Trajectory_Time' already exists, no changes made.")
df.to_csv("RequestsWithShortestPaths.csv", index=False)

In [11]:
requests_df = pd.read_csv("RequestsWithShortestPaths.csv")
waiting_df = pd.read_csv("waiting_times_node2.csv")

# Convert Request_ID to numeric
requests_df['Request_ID'] = requests_df['Request_ID'].astype(str).str.replace("Request_", "").astype(int)
waiting_df['RequestID'] = waiting_df['RequestID'].astype(int)

# Parse trajectory time as dictionaries
requests_df['Updated_Trajectory_Time'] = requests_df['Trajectory_Time'].apply(ast.literal_eval)

# Function to apply waiting time delay from the specified node
def apply_waiting_time_delay(traj_dict, delay_node, wait_minutes):
    new_traj = {}
    delay_applied = False
    delay = timedelta(minutes=wait_minutes)
    for key, value in traj_dict.items():
        start_str, end_str = value.split('-')
        start = datetime.strptime(start_str, "%H:%M:%S.%f")
        end = datetime.strptime(end_str, "%H:%M:%S.%f")
        if key == delay_node:
            # Only extend the END time of the node
            end += delay
            delay_applied = True
        elif delay_applied:
            # Apply delay to all subsequent start and end times
            start += delay
            end += delay
        new_traj[key] = f"{start.strftime('%H:%M:%S.%f')[:-3]}-{end.strftime('%H:%M:%S.%f')[:-3]}"
    return new_traj

# Apply the delay to each request
for _, row in waiting_df.iterrows():
    req_id = row['RequestID']
    node = row['NodeID']
    wait_time = row['WaitingTime_Minutes']
    match_idx = requests_df[requests_df['Request_ID'] == req_id].index
    if not match_idx.empty:
        idx = match_idx[0]
        traj_dict = requests_df.at[idx, 'Updated_Trajectory_Time']
        updated_traj = apply_waiting_time_delay(traj_dict, node, wait_time)
        requests_df.at[idx, 'Updated_Trajectory_Time'] = updated_traj

# Convert dicts to string before saving
requests_df['Updated_Trajectory_Time'] = requests_df['Updated_Trajectory_Time'].apply(str)
requests_df.to_csv("RequestsWithShortestPaths.csv", index=False)

#next Node started

In [12]:
requests_df = pd.read_csv('RequestsWithShortestPaths.csv')

# Parse Trajectory_Time column
def parse_trajectory_times(row):
    try:
        return ast.literal_eval(row['Updated_Trajectory_Time'])
    except:
        return {}

requests_df['Parsed_Times'] = requests_df.apply(parse_trajectory_times, axis=1)

# Collect info for each drone and node
log_data = []
for idx, row in requests_df.iterrows():
    drone_id = row['Drone_ID']
    request_id = row['Request_ID']
    times_dict = row['Parsed_Times']

    for node, time_range in times_dict.items():
        if node.startswith("Node_"):
            try:
                start_str, end_str = time_range.split('-')
                start_time = datetime.strptime(start_str, "%H:%M:%S.%f")
                end_time = datetime.strptime(end_str, "%H:%M:%S.%f")
                recharge_time = round((end_time - start_time).total_seconds() / 60, 2)  # in minutes

                # If recharge time is exactly 1 minute, replace it with 0
                if recharge_time == 1.0:
                    recharge_time = 0

                log_data.append({
                    "Drone_ID": drone_id,
                    "RequestID": request_id,
                    "NodeID": node,
                    "ArrivalTime": start_time.strftime("%H:%M:%S"),
                    "RechargingTime": recharge_time
                })
            except Exception as e:
                print(f"Error processing node {node} for drone {drone_id}: {e}")

log_df = pd.DataFrame(log_data)
# Sort the DataFrame by NodeID and ArrivalTime
log_df.sort_values(by=["NodeID", "ArrivalTime"], inplace=True)
log_df.to_csv("chargingslot_requests_Full.csv", index=False)

In [13]:
df = pd.read_csv("chargingslot_requests_Full.csv") 
# Filter rows where RechargingTime > 0
filtered_df = df[df['RechargingTime'] > 0]
filtered_df.to_csv("chargingslot_requests_Full.csv", index=False)

In [14]:
paths_df = pd.read_csv("RequestsWithShortestPaths.csv")
charging_df = pd.read_csv("chargingslot_requests_Full.csv")

# Parse list from string
paths_df['Node_IDs_Visited'] = paths_df['Node_IDs_Visited'].apply(ast.literal_eval)

# Extract numeric request ID from string like 'Request_1212'
def extract_request_id(request_str):
    return int(str(request_str).split('_')[-1])

# Build mapping of (Drone_ID, RequestID) -> third visited node
third_node_map = {}
for _, row in paths_df.iterrows():
    drone_id = row['Drone_ID']
    request_id = extract_request_id(row['Request_ID'])
    visited_nodes = row['Node_IDs_Visited']
    if len(visited_nodes) >= 3:
        third_node_map[(drone_id, request_id)] = visited_nodes[2]

# Filter charging requests
filtered_charging_df = charging_df[
    charging_df.apply(
        lambda row: third_node_map.get((row['Drone_ID'], int(row['RequestID']))) == row['NodeID'],
        axis=1
    )
]
filtered_charging_df.to_csv("chargingslot_requests_3.csv", index=False)


In [15]:
df = pd.read_csv("chargingslot_requests_3.csv")

# Truncate seconds in the ArrivalTime column
def truncate_seconds(time_str):
    try:
        t = datetime.strptime(time_str, "%H:%M:%S")
        return t.replace(second=0).strftime("%H:%M:%S")
    except:
        return time_str  # fallback if parsing fails3
df['ArrivalTime'] = df['ArrivalTime'].apply(truncate_seconds)
df.to_csv("chargingslot_requests_3.csv", index=False)

In [16]:
df1 = pd.read_csv("chargingslot_requests_2.csv")
df2 = pd.read_csv("chargingslot_requests_3.csv")

# Concatenate both files
combined_df = pd.concat([df1, df2], ignore_index=True)
# Save combined data
combined_df.to_csv("chargingslot_requests_23.csv", index=False)

In [17]:
# Constants
hover_power_watts = 159.01
battery_capacity_wh = 68.1
T_full_minutes = 80
factor = 0.2
tau = T_full_minutes * factor
alpha = 1.0  # heuristic-based weight factor

# Load CSV files
nodes_df = pd.read_csv('node_resource_count.csv')
drone_requests_df = pd.read_csv('chargingslot_requests_23.csv')
requests_df = pd.read_csv("RequestsWithShortestPaths.csv")

# Convert ArrivalTime
start_date = datetime.strptime("2024-06-20 07:00", "%Y-%m-%d %H:%M")
drone_requests_df['ArrivalTime'] = drone_requests_df['ArrivalTime'].apply(
    lambda x: start_date + timedelta(minutes=int(x.split(":")[0]) * 60 + int(x.split(":")[1])) - timedelta(hours=7)
)

# Preprocess Request Data
requests_df['RequestID'] = requests_df['Request_ID']
requests_df['Battery_Consumption_Per_Segment (%)'] = requests_df['Battery_Consumption_Per_Segment (%)'].apply(ast.literal_eval)
requests_df['Node_IDs_Visited'] = requests_df['Node_IDs_Visited'].apply(ast.literal_eval)

# Battery lookup
battery_lookup = {}
for _, row in requests_df.iterrows():
    request_id = row['RequestID']
    nodes = row['Node_IDs_Visited']
    consumptions = row['Battery_Consumption_Per_Segment (%)']
    battery = 100
    battery_lookup[(request_id, nodes[0])] = battery
    for i in range(1, len(nodes)):
        battery -= consumptions[i - 1]
        battery_lookup[(request_id, nodes[i])] = battery

# Final SOC mapping
requests_df['Request_ID_Number'] = requests_df['Request_ID']
final_soc_map = requests_df.set_index('Request_ID_Number')['Final_SOC'].to_dict()

# Queues
queues_adjusted = {
    node: {"RechargingQueue": [], "WaitingQueue": [], "OverflowQueue": []}
    for node in drone_requests_df['NodeID'].unique()
}

default_recharging_queue_limit = 1
default_waiting_queue_limit = 2

events = []
status_records = []
hover_log = []
recharge_extension_log = []

current_time = start_date
end_time = max(drone_requests_df['ArrivalTime']) + timedelta(minutes=80)

while current_time <= end_time or any(
    queues_adjusted[node]['RechargingQueue'] or
    queues_adjusted[node]['WaitingQueue'] or
    queues_adjusted[node]['OverflowQueue']
    for node in queues_adjusted.keys()
):
    new_arrivals = drone_requests_df[drone_requests_df['ArrivalTime'] == current_time]
    for _, request in new_arrivals.iterrows():
        node_id = request['NodeID']
        arrival_time = request['ArrivalTime']
        recharging_time = request['RechargingTime']
        request_id = int(request['RequestID'])

        if recharging_time == 0:
            continue

        if node_id in nodes_df['NodeID'].values:
            recharging_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'RechargingQueueLimit'].values[0]
            waiting_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'WaitingQueueLimit'].values[0]
        else:
            recharging_queue_limit = default_recharging_queue_limit
            waiting_queue_limit = default_waiting_queue_limit

        if (len(queues_adjusted[node_id]['RechargingQueue']) < recharging_queue_limit and
            len(queues_adjusted[node_id]['WaitingQueue']) == 0 and
            len(queues_adjusted[node_id]['OverflowQueue']) == 0):
            queues_adjusted[node_id]['RechargingQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "RechargingQueue (Direct Arrival - All Queues Empty)"))
        elif len(queues_adjusted[node_id]['WaitingQueue']) < waiting_queue_limit:
            queues_adjusted[node_id]['WaitingQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "WaitingQueue (New Arrival)"))
        else:
            queues_adjusted[node_id]['OverflowQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "OverflowQueue (New Arrival)"))

    for node_id in queues_adjusted.keys():
        if node_id in nodes_df['NodeID'].values:
            recharging_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'RechargingQueueLimit'].values[0]
            waiting_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'WaitingQueueLimit'].values[0]
        else:
            recharging_queue_limit = default_recharging_queue_limit
            waiting_queue_limit = default_waiting_queue_limit

        # Remove drones done recharging
        for req in queues_adjusted[node_id]['RechargingQueue'][:]:
            request_id, start_time, recharge = req
            if current_time >= start_time + timedelta(minutes=recharge):
                queues_adjusted[node_id]['RechargingQueue'].remove(req)
                events.append((current_time, request_id, "Released from RechargingQueue"))

        # heuristic-based promotion to Recharging Queue
        combined = []
        for request_id, arrival, recharge in queues_adjusted[node_id]['WaitingQueue']:
            wait_time = (current_time - arrival).total_seconds() / 60
            score = (wait_time ** 1.2) + alpha * recharge
            combined.append(("Waiting", request_id, arrival, recharge, recharge, score))

        for request_id, arrival, recharge in queues_adjusted[node_id]['OverflowQueue']:
            hover_time = (current_time - arrival).total_seconds() / 60
            battery_hover_pct = (hover_power_watts * hover_time / 60) / battery_capacity_wh * 100
            arrival_battery = battery_lookup.get((request_id, node_id), None)
            final_soc = final_soc_map.get(request_id, None)

            if arrival_battery is not None and final_soc is not None:
                total_pct = battery_hover_pct + arrival_battery
                if total_pct > 100:
                    Si = 0.11
                    Sf = final_soc
                else:
                    Si = 1 - (arrival_battery / 100)
                    Sf = Si + (battery_hover_pct / 100)
                if Sf >= 1 or Si >= Sf:
                    hover_recharge_time = 0
                else:
                    hover_recharge_time = -tau * math.log((1 - Sf) / (1 - Si))
                updated_recharge = recharge + hover_recharge_time
            else:
                updated_recharge = recharge

            wait_time = (current_time - arrival).total_seconds() / 60
            score = (wait_time ** 1.2) + alpha * updated_recharge
            combined.append(("Overflow", request_id, arrival, recharge, updated_recharge, score))

        combined.sort(key=lambda x: x[5])

        slots_available = recharging_queue_limit - len(queues_adjusted[node_id]['RechargingQueue'])
        promotions = combined[:slots_available]
        leftovers = combined[slots_available:]

        queues_adjusted[node_id]['WaitingQueue'] = []
        queues_adjusted[node_id]['OverflowQueue'] = []

        for origin, request_id, arrival, original_recharge, updated_recharge, score in promotions:
            if origin == "Overflow" and updated_recharge != original_recharge:
                queues_adjusted[node_id]['RechargingQueue'].append((request_id, current_time, updated_recharge))
                hover_time = (current_time - arrival).total_seconds() / 60
                battery_hover_pct = (hover_power_watts * hover_time / 60) / battery_capacity_wh * 100
                arrival_battery = battery_lookup.get((request_id, node_id), None)
                final_soc = final_soc_map.get(request_id, None)
                recharge_extension_log.append((request_id, node_id, original_recharge, updated_recharge, updated_recharge - original_recharge))
                hover_log.append((request_id, hover_time, battery_hover_pct, arrival_battery, final_soc, updated_recharge))
            else:
                queues_adjusted[node_id]['RechargingQueue'].append((request_id, current_time, original_recharge))

            events.append((current_time, request_id, f"Moved to RechargingQueue (WFS from {origin})"))

        # Promote remaining to waiting or overflow
        for origin, request_id, arrival, recharge, _, score in leftovers:
            if len(queues_adjusted[node_id]['WaitingQueue']) < waiting_queue_limit:
                queues_adjusted[node_id]['WaitingQueue'].append((request_id, arrival, recharge))
                events.append((current_time, request_id, f"Moved to WaitingQueue (Priority from {origin})"))
            else:
                queues_adjusted[node_id]['OverflowQueue'].append((request_id, arrival, recharge))
                events.append((current_time, request_id, f"Remains in OverflowQueue (After Priority Sort)"))

        recharging_status = [req[0] for req in queues_adjusted[node_id]['RechargingQueue']]
        waiting_status = [req[0] for req in queues_adjusted[node_id]['WaitingQueue']]
        overflow_status = [req[0] for req in queues_adjusted[node_id]['OverflowQueue']]
        status_records.append((current_time, node_id, recharging_status, waiting_status, overflow_status))

    current_time += timedelta(minutes=1)

status_df = pd.DataFrame(status_records, columns=["Time", "NodeID", "RechargingQueue", "WaitingQueue", "OverflowQueue"])
hover_df = pd.DataFrame(hover_log, columns=["Request_ID", "HoveringTime_Minutes", "Battery_Hover%", "BatteryLevelOnArrival", "Final_SOC", "UpdatedRechargeTime"])
recharge_df = pd.DataFrame(recharge_extension_log, columns=["Request_ID", "Node_ID", "Original_Recharge", "Updated_Recharge", "Increase"])
status_df.sort_values(by=["NodeID", "Time"], inplace=True)
status_df.to_csv("node_status_3.csv", index=False)
hover_df.to_csv("hover_log2.csv", index=False)
recharge_df.to_csv("recharge_time_increase2.csv", index=False)

In [18]:
status_df = pd.read_csv('node_status_3.csv')
requests_df = pd.read_csv('chargingslot_requests_23.csv')

# Convert ArrivalTime to datetime (assuming format HH:MM:SS)
requests_df['ArrivalTime'] = requests_df['ArrivalTime'].apply(
    lambda x: datetime.strptime(f"2024-06-20 {x}", "%Y-%m-%d %H:%M:%S")
    if isinstance(x, str) and len(x) == 8 else pd.to_datetime(x)
)

# Create lookup from (RequestID, NodeID) → ArrivalTime
arrival_lookup = requests_df.set_index(['RequestID', 'NodeID'])['ArrivalTime'].to_dict()

# Track first timestamp a drone appears in RechargingQueue
recharging_log = {}
for _, row in status_df.iterrows():
    time = pd.to_datetime(row['Time'])
    node_id = row['NodeID']
    try:
        recharging_q = eval(row['RechargingQueue'])  # Convert string to list
    except Exception:
        continue
    for request_id in recharging_q:
        key = (request_id, node_id)
        if key not in recharging_log:
            recharging_log[key] = time

# Prepare output with correct timestamps
waiting_records = []
for (request_id, node_id), recharge_start_time in recharging_log.items():
    key = (request_id, node_id)
    if key in arrival_lookup:
        arrival_time = arrival_lookup[key]
        waiting_time = (recharge_start_time - arrival_time).total_seconds() / 60
        waiting_records.append({
            'RequestID': request_id,
            'NodeID': node_id,
            'ArrivalTime': arrival_time,
            'RechargingStartTime': recharge_start_time,
            'WaitingTime_Minutes': round(waiting_time, 2)
        })

waiting_df = pd.DataFrame(waiting_records)
waiting_df.sort_values(by=['NodeID', 'ArrivalTime'], inplace=True)
waiting_df.to_csv('waiting_times_node23.csv', index=False)

In [19]:
file_path = "RequestsWithShortestPaths.csv"
df = pd.read_csv(file_path)

# Check and create Updated_Trajectory_Time column
if 'Updated_Trajectory_Time23' not in df.columns:
    df['Updated_Trajectory_Time23'] = df['Trajectory_Time']
else:
    print("'Updated_Trajectory_Time23' already exists, no changes made.")
df.to_csv("RequestsWithShortestPaths.csv", index=False)

In [20]:
requests_df = pd.read_csv("RequestsWithShortestPaths.csv")
waiting_df = pd.read_csv("waiting_times_node23.csv")

# Convert Request_ID to numeric
requests_df['Request_ID'] = requests_df['Request_ID'].astype(str).str.replace("Request_", "").astype(int)
waiting_df['RequestID'] = waiting_df['RequestID'].astype(int)

# Parse trajectory time as dictionaries
requests_df['Updated_Trajectory_Time23'] = requests_df['Trajectory_Time'].apply(ast.literal_eval)

# Function to apply waiting time delay from the specified node
def apply_waiting_time_delay(traj_dict, delay_node, wait_minutes):
    new_traj = {}
    delay_applied = False
    delay = timedelta(minutes=wait_minutes)
    for key, value in traj_dict.items():
        start_str, end_str = value.split('-')
        start = datetime.strptime(start_str, "%H:%M:%S.%f")
        end = datetime.strptime(end_str, "%H:%M:%S.%f")
        if key == delay_node:
            # Only extend the END time of the node
            end += delay
            delay_applied = True
        elif delay_applied:
            # Apply delay to all subsequent start and end times
            start += delay
            end += delay
        new_traj[key] = f"{start.strftime('%H:%M:%S.%f')[:-3]}-{end.strftime('%H:%M:%S.%f')[:-3]}"
    return new_traj

# Apply the delay to each request
for _, row in waiting_df.iterrows():
    req_id = row['RequestID']
    node = row['NodeID']
    wait_time = row['WaitingTime_Minutes']
    match_idx = requests_df[requests_df['Request_ID'] == req_id].index
    if not match_idx.empty:
        idx = match_idx[0]
        traj_dict = requests_df.at[idx, 'Updated_Trajectory_Time23']
        updated_traj = apply_waiting_time_delay(traj_dict, node, wait_time)
        requests_df.at[idx, 'Updated_Trajectory_Time23'] = updated_traj

# Convert dicts to string before saving
requests_df['Updated_Trajectory_Time23'] = requests_df['Updated_Trajectory_Time23'].apply(str)
requests_df.to_csv("RequestsWithShortestPaths.csv", index=False)

Processing the fourth node visited by the drones. The delay is propagated to the fourth node based on the impact of interferences at the previous node.

In [21]:
requests_df = pd.read_csv('RequestsWithShortestPaths.csv')

# Parse Trajectory_Time column
def parse_trajectory_times(row):
    try:
        return ast.literal_eval(row['Updated_Trajectory_Time23'])
    except:
        return {}

requests_df['Parsed_Times'] = requests_df.apply(parse_trajectory_times, axis=1)

# Collect info for each drone and node
log_data = []
for idx, row in requests_df.iterrows():
    drone_id = row['Drone_ID']
    request_id = row['Request_ID']
    times_dict = row['Parsed_Times']

    for node, time_range in times_dict.items():
        if node.startswith("Node_"):
            try:
                start_str, end_str = time_range.split('-')
                start_time = datetime.strptime(start_str, "%H:%M:%S.%f")
                end_time = datetime.strptime(end_str, "%H:%M:%S.%f")
                recharge_time = round((end_time - start_time).total_seconds() / 60, 2)  # in minutes

                # If recharge time is exactly 1 minute, replace it with 0
                if recharge_time == 1.0:
                    recharge_time = 0

                log_data.append({
                    "Drone_ID": drone_id,
                    "RequestID": request_id,
                    "NodeID": node,
                    "ArrivalTime": start_time.strftime("%H:%M:%S"),
                    "RechargingTime": recharge_time
                })
            except Exception as e:
                print(f"Error processing node {node} for drone {drone_id}: {e}")

log_df = pd.DataFrame(log_data)
# Sort the DataFrame by NodeID and ArrivalTime
log_df.sort_values(by=["NodeID", "ArrivalTime"], inplace=True)
log_df.to_csv("chargingslot_requests_Full.csv", index=False)

In [22]:
df = pd.read_csv("chargingslot_requests_Full.csv") 
# Filter rows where RechargingTime > 0
filtered_df = df[df['RechargingTime'] > 0]
filtered_df.to_csv("chargingslot_requests_Full.csv", index=False)

In [23]:
paths_df = pd.read_csv("RequestsWithShortestPaths.csv")
charging_df = pd.read_csv("chargingslot_requests_Full.csv")

# Parse list from string
paths_df['Node_IDs_Visited'] = paths_df['Node_IDs_Visited'].apply(ast.literal_eval)

# Extract numeric request ID from string like 'Request_1212'
def extract_request_id(request_str):
    return int(str(request_str).split('_')[-1])

# Build mapping of (Drone_ID, RequestID) -> fourth visited node
fourth_node_map = {}
for _, row in paths_df.iterrows():
    drone_id = row['Drone_ID']
    request_id = extract_request_id(row['Request_ID'])
    visited_nodes = row['Node_IDs_Visited']
    if len(visited_nodes) >= 4:
        fourth_node_map[(drone_id, request_id)] = visited_nodes[3]

# Filter charging requests
filtered_charging_df = charging_df[
    charging_df.apply(
        lambda row: fourth_node_map.get((row['Drone_ID'], int(row['RequestID']))) == row['NodeID'],
        axis=1
    )
]
filtered_charging_df.to_csv("chargingslot_requests_4.csv", index=False)

In [24]:
df = pd.read_csv("chargingslot_requests_4.csv")

# Truncate seconds in the ArrivalTime column
def truncate_seconds(time_str):
    try:
        t = datetime.strptime(time_str, "%H:%M:%S")
        return t.replace(second=0).strftime("%H:%M:%S")
    except:
        return time_str  # fallback if parsing fails3
df['ArrivalTime'] = df['ArrivalTime'].apply(truncate_seconds)
df.to_csv("chargingslot_requests_4.csv", index=False)

In [25]:
df1 = pd.read_csv("chargingslot_requests_23.csv")
df2 = pd.read_csv("chargingslot_requests_4.csv")

# Concatenate both files
combined_df = pd.concat([df1, df2], ignore_index=True)
# Save combined data
combined_df.to_csv("chargingslot_requests_234.csv", index=False)

C:\Users\sriz4651\AppData\Local\Temp\ipykernel_3324\4117994592.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([df1, df2], ignore_index=True)


In [26]:
# Constants
hover_power_watts = 159.01
battery_capacity_wh = 68.1
T_full_minutes = 80
factor = 0.2
tau = T_full_minutes * factor
alpha = 1.0  # heuristic-based weight factor

nodes_df = pd.read_csv('node_resource_count.csv')
drone_requests_df = pd.read_csv('chargingslot_requests_234.csv')
requests_df = pd.read_csv("RequestsWithShortestPaths.csv")

# Convert ArrivalTime
start_date = datetime.strptime("2024-06-20 07:00", "%Y-%m-%d %H:%M")
drone_requests_df['ArrivalTime'] = drone_requests_df['ArrivalTime'].apply(
    lambda x: start_date + timedelta(minutes=int(x.split(":")[0]) * 60 + int(x.split(":")[1])) - timedelta(hours=7)
)

# Preprocess Request Data
requests_df['RequestID'] = requests_df['Request_ID']
requests_df['Battery_Consumption_Per_Segment (%)'] = requests_df['Battery_Consumption_Per_Segment (%)'].apply(ast.literal_eval)
requests_df['Node_IDs_Visited'] = requests_df['Node_IDs_Visited'].apply(ast.literal_eval)

# Battery lookup
battery_lookup = {}
for _, row in requests_df.iterrows():
    request_id = row['RequestID']
    nodes = row['Node_IDs_Visited']
    consumptions = row['Battery_Consumption_Per_Segment (%)']
    battery = 100
    battery_lookup[(request_id, nodes[0])] = battery
    for i in range(1, len(nodes)):
        battery -= consumptions[i - 1]
        battery_lookup[(request_id, nodes[i])] = battery

# Final SOC mapping
requests_df['Request_ID_Number'] = requests_df['Request_ID']
final_soc_map = requests_df.set_index('Request_ID_Number')['Final_SOC'].to_dict()

# Queues
queues_adjusted = {
    node: {"RechargingQueue": [], "WaitingQueue": [], "OverflowQueue": []}
    for node in drone_requests_df['NodeID'].unique()
}

default_recharging_queue_limit = 1
default_waiting_queue_limit = 2

events = []
status_records = []
hover_log = []
recharge_extension_log = []

current_time = start_date
end_time = max(drone_requests_df['ArrivalTime']) + timedelta(minutes=80)

while current_time <= end_time or any(
    queues_adjusted[node]['RechargingQueue'] or
    queues_adjusted[node]['WaitingQueue'] or
    queues_adjusted[node]['OverflowQueue']
    for node in queues_adjusted.keys()
):
    new_arrivals = drone_requests_df[drone_requests_df['ArrivalTime'] == current_time]
    for _, request in new_arrivals.iterrows():
        node_id = request['NodeID']
        arrival_time = request['ArrivalTime']
        recharging_time = request['RechargingTime']
        request_id = int(request['RequestID'])

        if recharging_time == 0:
            continue

        if node_id in nodes_df['NodeID'].values:
            recharging_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'RechargingQueueLimit'].values[0]
            waiting_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'WaitingQueueLimit'].values[0]
        else:
            recharging_queue_limit = default_recharging_queue_limit
            waiting_queue_limit = default_waiting_queue_limit

        if (len(queues_adjusted[node_id]['RechargingQueue']) < recharging_queue_limit and
            len(queues_adjusted[node_id]['WaitingQueue']) == 0 and
            len(queues_adjusted[node_id]['OverflowQueue']) == 0):
            queues_adjusted[node_id]['RechargingQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "RechargingQueue (Direct Arrival - All Queues Empty)"))
        elif len(queues_adjusted[node_id]['WaitingQueue']) < waiting_queue_limit:
            queues_adjusted[node_id]['WaitingQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "WaitingQueue (New Arrival)"))
        else:
            queues_adjusted[node_id]['OverflowQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "OverflowQueue (New Arrival)"))

    for node_id in queues_adjusted.keys():
        if node_id in nodes_df['NodeID'].values:
            recharging_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'RechargingQueueLimit'].values[0]
            waiting_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'WaitingQueueLimit'].values[0]
        else:
            recharging_queue_limit = default_recharging_queue_limit
            waiting_queue_limit = default_waiting_queue_limit

        # Remove drones done recharging
        for req in queues_adjusted[node_id]['RechargingQueue'][:]:
            request_id, start_time, recharge = req
            if current_time >= start_time + timedelta(minutes=recharge):
                queues_adjusted[node_id]['RechargingQueue'].remove(req)
                events.append((current_time, request_id, "Released from RechargingQueue"))

        # heuristic-based promotion to Recharging
        combined = []
        for request_id, arrival, recharge in queues_adjusted[node_id]['WaitingQueue']:
            wait_time = (current_time - arrival).total_seconds() / 60
            score = (wait_time ** 1.2) + alpha * recharge
            combined.append(("Waiting", request_id, arrival, recharge, recharge, score))

        for request_id, arrival, recharge in queues_adjusted[node_id]['OverflowQueue']:
            hover_time = (current_time - arrival).total_seconds() / 60
            battery_hover_pct = (hover_power_watts * hover_time / 60) / battery_capacity_wh * 100
            arrival_battery = battery_lookup.get((request_id, node_id), None)
            final_soc = final_soc_map.get(request_id, None)

            if arrival_battery is not None and final_soc is not None:
                total_pct = battery_hover_pct + arrival_battery
                if total_pct > 100:
                    Si = 0.11
                    Sf = final_soc
                else:
                    Si = 1 - (arrival_battery / 100)
                    Sf = Si + (battery_hover_pct / 100)
                if Sf >= 1 or Si >= Sf:
                    hover_recharge_time = 0
                else:
                    hover_recharge_time = -tau * math.log((1 - Sf) / (1 - Si))
                updated_recharge = recharge + hover_recharge_time
            else:
                updated_recharge = recharge

            wait_time = (current_time - arrival).total_seconds() / 60
            score = (wait_time ** 1.2) + alpha * updated_recharge
            combined.append(("Overflow", request_id, arrival, recharge, updated_recharge, score))

        combined.sort(key=lambda x: x[5])

        slots_available = recharging_queue_limit - len(queues_adjusted[node_id]['RechargingQueue'])
        promotions = combined[:slots_available]
        leftovers = combined[slots_available:]

        queues_adjusted[node_id]['WaitingQueue'] = []
        queues_adjusted[node_id]['OverflowQueue'] = []

        for origin, request_id, arrival, original_recharge, updated_recharge, score in promotions:
            if origin == "Overflow" and updated_recharge != original_recharge:
                queues_adjusted[node_id]['RechargingQueue'].append((request_id, current_time, updated_recharge))
                hover_time = (current_time - arrival).total_seconds() / 60
                battery_hover_pct = (hover_power_watts * hover_time / 60) / battery_capacity_wh * 100
                arrival_battery = battery_lookup.get((request_id, node_id), None)
                final_soc = final_soc_map.get(request_id, None)
                recharge_extension_log.append((request_id, node_id, original_recharge, updated_recharge, updated_recharge - original_recharge))
                hover_log.append((request_id, hover_time, battery_hover_pct, arrival_battery, final_soc, updated_recharge))
            else:
                queues_adjusted[node_id]['RechargingQueue'].append((request_id, current_time, original_recharge))

            events.append((current_time, request_id, f"Moved to RechargingQueue (WFS from {origin})"))

        # Promote remaining to waiting or overflow
        for origin, request_id, arrival, recharge, _, score in leftovers:
            if len(queues_adjusted[node_id]['WaitingQueue']) < waiting_queue_limit:
                queues_adjusted[node_id]['WaitingQueue'].append((request_id, arrival, recharge))
                events.append((current_time, request_id, f"Moved to WaitingQueue (Priority from {origin})"))
            else:
                queues_adjusted[node_id]['OverflowQueue'].append((request_id, arrival, recharge))
                events.append((current_time, request_id, f"Remains in OverflowQueue (After Priority Sort)"))

        recharging_status = [req[0] for req in queues_adjusted[node_id]['RechargingQueue']]
        waiting_status = [req[0] for req in queues_adjusted[node_id]['WaitingQueue']]
        overflow_status = [req[0] for req in queues_adjusted[node_id]['OverflowQueue']]
        status_records.append((current_time, node_id, recharging_status, waiting_status, overflow_status))

    current_time += timedelta(minutes=1)

status_df = pd.DataFrame(status_records, columns=["Time", "NodeID", "RechargingQueue", "WaitingQueue", "OverflowQueue"])
hover_df = pd.DataFrame(hover_log, columns=["Request_ID", "HoveringTime_Minutes", "Battery_Hover%", "BatteryLevelOnArrival", "Final_SOC", "UpdatedRechargeTime"])
recharge_df = pd.DataFrame(recharge_extension_log, columns=["Request_ID", "Node_ID", "Original_Recharge", "Updated_Recharge", "Increase"])
status_df.sort_values(by=["NodeID", "Time"], inplace=True)
status_df.to_csv("node_status_4.csv", index=False)
hover_df.to_csv("hover_log2.csv", index=False)
recharge_df.to_csv("recharge_time_increase2.csv", index=False)

In [27]:
status_df = pd.read_csv('node_status_4.csv')
requests_df = pd.read_csv('chargingslot_requests_234.csv')

# Convert ArrivalTime to datetime (assuming format HH:MM:SS)
requests_df['ArrivalTime'] = requests_df['ArrivalTime'].apply(
    lambda x: datetime.strptime(f"2024-06-20 {x}", "%Y-%m-%d %H:%M:%S")
    if isinstance(x, str) and len(x) == 8 else pd.to_datetime(x)
)

# Create lookup from (RequestID, NodeID) → ArrivalTime
arrival_lookup = requests_df.set_index(['RequestID', 'NodeID'])['ArrivalTime'].to_dict()

# Track first timestamp a drone appears in RechargingQueue
recharging_log = {}
for _, row in status_df.iterrows():
    time = pd.to_datetime(row['Time'])
    node_id = row['NodeID']
    try:
        recharging_q = eval(row['RechargingQueue'])  # Convert string to list
    except Exception:
        continue
    for request_id in recharging_q:
        key = (request_id, node_id)
        if key not in recharging_log:
            recharging_log[key] = time

# Prepare output with correct timestamps
waiting_records = []
for (request_id, node_id), recharge_start_time in recharging_log.items():
    key = (request_id, node_id)
    if key in arrival_lookup:
        arrival_time = arrival_lookup[key]
        waiting_time = (recharge_start_time - arrival_time).total_seconds() / 60
        waiting_records.append({
            'RequestID': request_id,
            'NodeID': node_id,
            'ArrivalTime': arrival_time,
            'RechargingStartTime': recharge_start_time,
            'WaitingTime_Minutes': round(waiting_time, 2)
        })

waiting_df = pd.DataFrame(waiting_records)
waiting_df.sort_values(by=['NodeID', 'ArrivalTime'], inplace=True)
waiting_df.to_csv('waiting_times_node234.csv', index=False)

In [28]:
file_path = "RequestsWithShortestPaths.csv"
df = pd.read_csv(file_path)

# Check and create Updated_Trajectory_Time column
if 'Updated_Trajectory_Time234' not in df.columns:
    df['Updated_Trajectory_Time234'] = df['Trajectory_Time']
else:
    print("'Updated_Trajectory_Time234' already exists, no changes made.")
df.to_csv("RequestsWithShortestPaths.csv", index=False)

In [29]:
requests_df = pd.read_csv("RequestsWithShortestPaths.csv")
waiting_df = pd.read_csv("waiting_times_node234.csv")

# Convert Request_ID to numeric
requests_df['Request_ID'] = requests_df['Request_ID'].astype(str).str.replace("Request_", "").astype(int)
waiting_df['RequestID'] = waiting_df['RequestID'].astype(int)

# Parse trajectory time as dictionaries
requests_df['Updated_Trajectory_Time234'] = requests_df['Trajectory_Time'].apply(ast.literal_eval)

# Function to apply waiting time delay from the specified node
def apply_waiting_time_delay(traj_dict, delay_node, wait_minutes):
    new_traj = {}
    delay_applied = False
    delay = timedelta(minutes=wait_minutes)
    for key, value in traj_dict.items():
        start_str, end_str = value.split('-')
        start = datetime.strptime(start_str, "%H:%M:%S.%f")
        end = datetime.strptime(end_str, "%H:%M:%S.%f")
        if key == delay_node:
            # Only extend the END time of the node
            end += delay
            delay_applied = True
        elif delay_applied:
            # Apply delay to all subsequent start and end times
            start += delay
            end += delay
        new_traj[key] = f"{start.strftime('%H:%M:%S.%f')[:-3]}-{end.strftime('%H:%M:%S.%f')[:-3]}"
    return new_traj

# Apply the delay to each request
for _, row in waiting_df.iterrows():
    req_id = row['RequestID']
    node = row['NodeID']
    wait_time = row['WaitingTime_Minutes']
    match_idx = requests_df[requests_df['Request_ID'] == req_id].index
    if not match_idx.empty:
        idx = match_idx[0]
        traj_dict = requests_df.at[idx, 'Updated_Trajectory_Time234']
        updated_traj = apply_waiting_time_delay(traj_dict, node, wait_time)
        requests_df.at[idx, 'Updated_Trajectory_Time234'] = updated_traj

requests_df['Updated_Trajectory_Time234'] = requests_df['Updated_Trajectory_Time234'].apply(str)
requests_df.to_csv("RequestsWithShortestPaths.csv", index=False)

Processing the fifth node visited by the drones. The delay is propagated to the fifth node based on the impact of interferences at the previous node.


In [30]:
requests_df = pd.read_csv('RequestsWithShortestPaths.csv')

# Parse Trajectory_Time column
def parse_trajectory_times(row):
    try:
        return ast.literal_eval(row['Updated_Trajectory_Time234'])
    except:
        return {}

requests_df['Parsed_Times'] = requests_df.apply(parse_trajectory_times, axis=1)

# Collect info for each drone and node
log_data = []
for idx, row in requests_df.iterrows():
    drone_id = row['Drone_ID']
    request_id = row['Request_ID']
    times_dict = row['Parsed_Times']

    for node, time_range in times_dict.items():
        if node.startswith("Node_"):
            try:
                start_str, end_str = time_range.split('-')
                start_time = datetime.strptime(start_str, "%H:%M:%S.%f")
                end_time = datetime.strptime(end_str, "%H:%M:%S.%f")
                recharge_time = round((end_time - start_time).total_seconds() / 60, 2)  # in minutes

                # If recharge time is exactly 1 minute, replace it with 0
                if recharge_time == 1.0:
                    recharge_time = 0

                log_data.append({
                    "Drone_ID": drone_id,
                    "RequestID": request_id,
                    "NodeID": node,
                    "ArrivalTime": start_time.strftime("%H:%M:%S"),
                    "RechargingTime": recharge_time
                })
            except Exception as e:
                print(f"Error processing node {node} for drone {drone_id}: {e}")

log_df = pd.DataFrame(log_data)
# Sort the DataFrame by NodeID and ArrivalTime
log_df.sort_values(by=["NodeID", "ArrivalTime"], inplace=True)
log_df.to_csv("chargingslot_requests_Full.csv", index=False)

In [31]:
df = pd.read_csv("chargingslot_requests_Full.csv") 
# Filter rows where RechargingTime > 0
filtered_df = df[df['RechargingTime'] > 0]
filtered_df.to_csv("chargingslot_requests_Full.csv", index=False)

In [32]:
paths_df = pd.read_csv("RequestsWithShortestPaths.csv")
charging_df = pd.read_csv("chargingslot_requests_Full.csv")

# Parse list from string
paths_df['Node_IDs_Visited'] = paths_df['Node_IDs_Visited'].apply(ast.literal_eval)

# Extract numeric request ID from string like 'Request_1212'
def extract_request_id(request_str):
    return int(str(request_str).split('_')[-1])

# Build mapping of (Drone_ID, RequestID) -> second visited node
fifth_node_map = {}
for _, row in paths_df.iterrows():
    drone_id = row['Drone_ID']
    request_id = extract_request_id(row['Request_ID'])
    visited_nodes = row['Node_IDs_Visited']
    if len(visited_nodes) >= 5:
        fifth_node_map[(drone_id, request_id)] = visited_nodes[4]

# Filter charging requests
filtered_charging_df = charging_df[
    charging_df.apply(
        lambda row: fifth_node_map.get((row['Drone_ID'], int(row['RequestID']))) == row['NodeID'],
        axis=1
    )
]
filtered_charging_df.to_csv("chargingslot_requests_5.csv", index=False)

In [33]:
df = pd.read_csv("chargingslot_requests_5.csv")

# Truncate seconds in the ArrivalTime column
def truncate_seconds(time_str):
    try:
        t = datetime.strptime(time_str, "%H:%M:%S")
        return t.replace(second=0).strftime("%H:%M:%S")
    except:
        return time_str  # fallback if parsing fails3
df['ArrivalTime'] = df['ArrivalTime'].apply(truncate_seconds)
df.to_csv("chargingslot_requests_5.csv", index=False)

In [34]:
df1 = pd.read_csv("chargingslot_requests_234.csv")
df2 = pd.read_csv("chargingslot_requests_5.csv")

# Concatenate both files
combined_df = pd.concat([df1, df2], ignore_index=True)
# Save combined data
combined_df.to_csv("chargingslot_requests_2345.csv", index=False)

C:\Users\sriz4651\AppData\Local\Temp\ipykernel_3324\2975635542.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([df1, df2], ignore_index=True)


In [35]:
# Constants
hover_power_watts = 159.01
battery_capacity_wh = 68.1
T_full_minutes = 80
factor = 0.2
tau = T_full_minutes * factor
alpha = 1.0  # heuristic-based weight factor

nodes_df = pd.read_csv('node_resource_count.csv')
drone_requests_df = pd.read_csv('chargingslot_requests_2345.csv')
requests_df = pd.read_csv("RequestsWithShortestPaths.csv")

# Convert ArrivalTime
start_date = datetime.strptime("2024-06-20 07:00", "%Y-%m-%d %H:%M")
drone_requests_df['ArrivalTime'] = drone_requests_df['ArrivalTime'].apply(
    lambda x: start_date + timedelta(minutes=int(x.split(":")[0]) * 60 + int(x.split(":")[1])) - timedelta(hours=7)
)

# Preprocess Request Data
requests_df['RequestID'] = requests_df['Request_ID']
requests_df['Battery_Consumption_Per_Segment (%)'] = requests_df['Battery_Consumption_Per_Segment (%)'].apply(ast.literal_eval)
requests_df['Node_IDs_Visited'] = requests_df['Node_IDs_Visited'].apply(ast.literal_eval)

# Battery lookup
battery_lookup = {}
for _, row in requests_df.iterrows():
    request_id = row['RequestID']
    nodes = row['Node_IDs_Visited']
    consumptions = row['Battery_Consumption_Per_Segment (%)']
    battery = 100
    battery_lookup[(request_id, nodes[0])] = battery
    for i in range(1, len(nodes)):
        battery -= consumptions[i - 1]
        battery_lookup[(request_id, nodes[i])] = battery

# Final SOC mapping
requests_df['Request_ID_Number'] = requests_df['Request_ID']
final_soc_map = requests_df.set_index('Request_ID_Number')['Final_SOC'].to_dict()

# Queues
queues_adjusted = {
    node: {"RechargingQueue": [], "WaitingQueue": [], "OverflowQueue": []}
    for node in drone_requests_df['NodeID'].unique()
}

default_recharging_queue_limit = 1
default_waiting_queue_limit = 2

events = []
status_records = []
hover_log = []
recharge_extension_log = []

current_time = start_date
end_time = max(drone_requests_df['ArrivalTime']) + timedelta(minutes=80)

while current_time <= end_time or any(
    queues_adjusted[node]['RechargingQueue'] or
    queues_adjusted[node]['WaitingQueue'] or
    queues_adjusted[node]['OverflowQueue']
    for node in queues_adjusted.keys()
):
    new_arrivals = drone_requests_df[drone_requests_df['ArrivalTime'] == current_time]
    for _, request in new_arrivals.iterrows():
        node_id = request['NodeID']
        arrival_time = request['ArrivalTime']
        recharging_time = request['RechargingTime']
        request_id = int(request['RequestID'])

        if recharging_time == 0:
            continue

        if node_id in nodes_df['NodeID'].values:
            recharging_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'RechargingQueueLimit'].values[0]
            waiting_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'WaitingQueueLimit'].values[0]
        else:
            recharging_queue_limit = default_recharging_queue_limit
            waiting_queue_limit = default_waiting_queue_limit

        if (len(queues_adjusted[node_id]['RechargingQueue']) < recharging_queue_limit and
            len(queues_adjusted[node_id]['WaitingQueue']) == 0 and
            len(queues_adjusted[node_id]['OverflowQueue']) == 0):
            queues_adjusted[node_id]['RechargingQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "RechargingQueue (Direct Arrival - All Queues Empty)"))
        elif len(queues_adjusted[node_id]['WaitingQueue']) < waiting_queue_limit:
            queues_adjusted[node_id]['WaitingQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "WaitingQueue (New Arrival)"))
        else:
            queues_adjusted[node_id]['OverflowQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "OverflowQueue (New Arrival)"))

    for node_id in queues_adjusted.keys():
        if node_id in nodes_df['NodeID'].values:
            recharging_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'RechargingQueueLimit'].values[0]
            waiting_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'WaitingQueueLimit'].values[0]
        else:
            recharging_queue_limit = default_recharging_queue_limit
            waiting_queue_limit = default_waiting_queue_limit

        # Remove drones done recharging
        for req in queues_adjusted[node_id]['RechargingQueue'][:]:
            request_id, start_time, recharge = req
            if current_time >= start_time + timedelta(minutes=recharge):
                queues_adjusted[node_id]['RechargingQueue'].remove(req)
                events.append((current_time, request_id, "Released from RechargingQueue"))

        # heusristic-based promotion to Recharging
        combined = []
        for request_id, arrival, recharge in queues_adjusted[node_id]['WaitingQueue']:
            wait_time = (current_time - arrival).total_seconds() / 60
            score = (wait_time ** 1.2) + alpha * recharge
            combined.append(("Waiting", request_id, arrival, recharge, recharge, score))

        for request_id, arrival, recharge in queues_adjusted[node_id]['OverflowQueue']:
            hover_time = (current_time - arrival).total_seconds() / 60
            battery_hover_pct = (hover_power_watts * hover_time / 60) / battery_capacity_wh * 100
            arrival_battery = battery_lookup.get((request_id, node_id), None)
            final_soc = final_soc_map.get(request_id, None)

            if arrival_battery is not None and final_soc is not None:
                total_pct = battery_hover_pct + arrival_battery
                if total_pct > 100:
                    Si = 0.11
                    Sf = final_soc
                else:
                    Si = 1 - (arrival_battery / 100)
                    Sf = Si + (battery_hover_pct / 100)
                if Sf >= 1 or Si >= Sf:
                    hover_recharge_time = 0
                else:
                    hover_recharge_time = -tau * math.log((1 - Sf) / (1 - Si))
                updated_recharge = recharge + hover_recharge_time
            else:
                updated_recharge = recharge

            wait_time = (current_time - arrival).total_seconds() / 60
            score = (wait_time ** 1.2) + alpha * updated_recharge
            combined.append(("Overflow", request_id, arrival, recharge, updated_recharge, score))

        combined.sort(key=lambda x: x[5])

        slots_available = recharging_queue_limit - len(queues_adjusted[node_id]['RechargingQueue'])
        promotions = combined[:slots_available]
        leftovers = combined[slots_available:]

        queues_adjusted[node_id]['WaitingQueue'] = []
        queues_adjusted[node_id]['OverflowQueue'] = []

        for origin, request_id, arrival, original_recharge, updated_recharge, score in promotions:
            if origin == "Overflow" and updated_recharge != original_recharge:
                queues_adjusted[node_id]['RechargingQueue'].append((request_id, current_time, updated_recharge))
                hover_time = (current_time - arrival).total_seconds() / 60
                battery_hover_pct = (hover_power_watts * hover_time / 60) / battery_capacity_wh * 100
                arrival_battery = battery_lookup.get((request_id, node_id), None)
                final_soc = final_soc_map.get(request_id, None)
                recharge_extension_log.append((request_id, node_id, original_recharge, updated_recharge, updated_recharge - original_recharge))
                hover_log.append((request_id, hover_time, battery_hover_pct, arrival_battery, final_soc, updated_recharge))
            else:
                queues_adjusted[node_id]['RechargingQueue'].append((request_id, current_time, original_recharge))

            events.append((current_time, request_id, f"Moved to RechargingQueue (WFS from {origin})"))

        # Promote remaining to waiting or overflow
        for origin, request_id, arrival, recharge, _, score in leftovers:
            if len(queues_adjusted[node_id]['WaitingQueue']) < waiting_queue_limit:
                queues_adjusted[node_id]['WaitingQueue'].append((request_id, arrival, recharge))
                events.append((current_time, request_id, f"Moved to WaitingQueue (Priority from {origin})"))
            else:
                queues_adjusted[node_id]['OverflowQueue'].append((request_id, arrival, recharge))
                events.append((current_time, request_id, f"Remains in OverflowQueue (After Priority Sort)"))

        recharging_status = [req[0] for req in queues_adjusted[node_id]['RechargingQueue']]
        waiting_status = [req[0] for req in queues_adjusted[node_id]['WaitingQueue']]
        overflow_status = [req[0] for req in queues_adjusted[node_id]['OverflowQueue']]
        status_records.append((current_time, node_id, recharging_status, waiting_status, overflow_status))

    current_time += timedelta(minutes=1)

status_df = pd.DataFrame(status_records, columns=["Time", "NodeID", "RechargingQueue", "WaitingQueue", "OverflowQueue"])
hover_df = pd.DataFrame(hover_log, columns=["Request_ID", "HoveringTime_Minutes", "Battery_Hover%", "BatteryLevelOnArrival", "Final_SOC", "UpdatedRechargeTime"])
recharge_df = pd.DataFrame(recharge_extension_log, columns=["Request_ID", "Node_ID", "Original_Recharge", "Updated_Recharge", "Increase"])
status_df.sort_values(by=["NodeID", "Time"], inplace=True)
status_df.to_csv("node_status_5.csv", index=False)
hover_df.to_csv("hover_log2.csv", index=False)
recharge_df.to_csv("recharge_time_increase2.csv", index=False)

In [36]:
status_df = pd.read_csv('node_status_5.csv')
requests_df = pd.read_csv('chargingslot_requests_2345.csv')

# Convert ArrivalTime to datetime (assuming format HH:MM:SS)
requests_df['ArrivalTime'] = requests_df['ArrivalTime'].apply(
    lambda x: datetime.strptime(f"2024-06-20 {x}", "%Y-%m-%d %H:%M:%S")
    if isinstance(x, str) and len(x) == 8 else pd.to_datetime(x)
)

# Create lookup from (RequestID, NodeID) → ArrivalTime
arrival_lookup = requests_df.set_index(['RequestID', 'NodeID'])['ArrivalTime'].to_dict()

# Track first timestamp a drone appears in RechargingQueue
recharging_log = {}
for _, row in status_df.iterrows():
    time = pd.to_datetime(row['Time'])
    node_id = row['NodeID']
    try:
        recharging_q = eval(row['RechargingQueue'])  # Convert string to list
    except Exception:
        continue
    for request_id in recharging_q:
        key = (request_id, node_id)
        if key not in recharging_log:
            recharging_log[key] = time

# Prepare output with correct timestamps
waiting_records = []
for (request_id, node_id), recharge_start_time in recharging_log.items():
    key = (request_id, node_id)
    if key in arrival_lookup:
        arrival_time = arrival_lookup[key]
        waiting_time = (recharge_start_time - arrival_time).total_seconds() / 60
        waiting_records.append({
            'RequestID': request_id,
            'NodeID': node_id,
            'ArrivalTime': arrival_time,
            'RechargingStartTime': recharge_start_time,
            'WaitingTime_Minutes': round(waiting_time, 2)
        })

waiting_df = pd.DataFrame(waiting_records)
waiting_df.sort_values(by=['NodeID', 'ArrivalTime'], inplace=True)
waiting_df.to_csv('waiting_times_node2345.csv', index=False)

In [37]:
file_path = "RequestsWithShortestPaths.csv"
df = pd.read_csv(file_path)

# Check and create Updated_Trajectory_Time column
if 'Updated_Trajectory_Time2345' not in df.columns:
    df['Updated_Trajectory_Time2345'] = df['Trajectory_Time']
else:
    print("'Updated_Trajectory_Time2345' already exists, no changes made.")
df.to_csv("RequestsWithShortestPaths.csv", index=False)

In [38]:
requests_df = pd.read_csv("RequestsWithShortestPaths.csv")
waiting_df = pd.read_csv("waiting_times_node2345.csv")

# Convert Request_ID to numeric
requests_df['Request_ID'] = requests_df['Request_ID'].astype(str).str.replace("Request_", "").astype(int)
waiting_df['RequestID'] = waiting_df['RequestID'].astype(int)

# Parse trajectory time as dictionaries
requests_df['Updated_Trajectory_Time2345'] = requests_df['Trajectory_Time'].apply(ast.literal_eval)

# Function to apply waiting time delay from the specified node
def apply_waiting_time_delay(traj_dict, delay_node, wait_minutes):
    new_traj = {}
    delay_applied = False
    delay = timedelta(minutes=wait_minutes)
    for key, value in traj_dict.items():
        start_str, end_str = value.split('-')
        start = datetime.strptime(start_str, "%H:%M:%S.%f")
        end = datetime.strptime(end_str, "%H:%M:%S.%f")
        if key == delay_node:
            # Only extend the END time of the node
            end += delay
            delay_applied = True
        elif delay_applied:
            # Apply delay to all subsequent start and end times
            start += delay
            end += delay
        new_traj[key] = f"{start.strftime('%H:%M:%S.%f')[:-3]}-{end.strftime('%H:%M:%S.%f')[:-3]}"
    return new_traj

# Apply the delay to each request
for _, row in waiting_df.iterrows():
    req_id = row['RequestID']
    node = row['NodeID']
    wait_time = row['WaitingTime_Minutes']
    match_idx = requests_df[requests_df['Request_ID'] == req_id].index
    if not match_idx.empty:
        idx = match_idx[0]
        traj_dict = requests_df.at[idx, 'Updated_Trajectory_Time2345']
        updated_traj = apply_waiting_time_delay(traj_dict, node, wait_time)
        requests_df.at[idx, 'Updated_Trajectory_Time2345'] = updated_traj

# Convert dicts to string before saving
requests_df['Updated_Trajectory_Time2345'] = requests_df['Updated_Trajectory_Time2345'].apply(str)
requests_df.to_csv("RequestsWithShortestPaths.csv", index=False)

Processing the sixth node visited by the drones. The delay is propagated to the sixth node based on the impact of interferences at the previous node.

In [39]:
requests_df = pd.read_csv('RequestsWithShortestPaths.csv')

# Parse Trajectory_Time column
def parse_trajectory_times(row):
    try:
        return ast.literal_eval(row['Updated_Trajectory_Time2345'])
    except:
        return {}

requests_df['Parsed_Times'] = requests_df.apply(parse_trajectory_times, axis=1)

# Collect info for each drone and node
log_data = []
for idx, row in requests_df.iterrows():
    drone_id = row['Drone_ID']
    request_id = row['Request_ID']
    times_dict = row['Parsed_Times']

    for node, time_range in times_dict.items():
        if node.startswith("Node_"):
            try:
                start_str, end_str = time_range.split('-')
                start_time = datetime.strptime(start_str, "%H:%M:%S.%f")
                end_time = datetime.strptime(end_str, "%H:%M:%S.%f")
                recharge_time = round((end_time - start_time).total_seconds() / 60, 2)  # in minutes

                # If recharge time is exactly 1 minute, replace it with 0
                if recharge_time == 1.0:
                    recharge_time = 0

                log_data.append({
                    "Drone_ID": drone_id,
                    "RequestID": request_id,
                    "NodeID": node,
                    "ArrivalTime": start_time.strftime("%H:%M:%S"),
                    "RechargingTime": recharge_time
                })
            except Exception as e:
                print(f"Error processing node {node} for drone {drone_id}: {e}")

log_df = pd.DataFrame(log_data)
# Sort the DataFrame by NodeID and ArrivalTime
log_df.sort_values(by=["NodeID", "ArrivalTime"], inplace=True)
log_df.to_csv("chargingslot_requests_Full.csv", index=False)

In [40]:
df = pd.read_csv("chargingslot_requests_Full.csv")  
# Filter rows where RechargingTime > 0
filtered_df = df[df['RechargingTime'] > 0]
filtered_df.to_csv("chargingslot_requests_Full.csv", index=False)

In [41]:
paths_df = pd.read_csv("RequestsWithShortestPaths.csv")
charging_df = pd.read_csv("chargingslot_requests_Full.csv")

# Parse list from string
paths_df['Node_IDs_Visited'] = paths_df['Node_IDs_Visited'].apply(ast.literal_eval)

# Extract numeric request ID from string like 'Request_1212'
def extract_request_id(request_str):
    return int(str(request_str).split('_')[-1])

# Build mapping of (Drone_ID, RequestID) -> sixth visited node
sixth_node_map = {}
for _, row in paths_df.iterrows():
    drone_id = row['Drone_ID']
    request_id = extract_request_id(row['Request_ID'])
    visited_nodes = row['Node_IDs_Visited']
    if len(visited_nodes) >= 6:
        sixth_node_map[(drone_id, request_id)] = visited_nodes[5]

# Filter charging requests
filtered_charging_df = charging_df[
    charging_df.apply(
        lambda row: sixth_node_map.get((row['Drone_ID'], int(row['RequestID']))) == row['NodeID'],
        axis=1
    )
]
filtered_charging_df.to_csv("chargingslot_requests_6.csv", index=False)


In [42]:
df = pd.read_csv("chargingslot_requests_6.csv")

# Truncate seconds in the ArrivalTime column
def truncate_seconds(time_str):
    try:
        t = datetime.strptime(time_str, "%H:%M:%S")
        return t.replace(second=0).strftime("%H:%M:%S")
    except:
        return time_str  # fallback if parsing fails3
df['ArrivalTime'] = df['ArrivalTime'].apply(truncate_seconds)
df.to_csv("chargingslot_requests_6.csv", index=False)

In [43]:
df1 = pd.read_csv("chargingslot_requests_2345.csv")
df2 = pd.read_csv("chargingslot_requests_6.csv")

# Concatenate both files
combined_df = pd.concat([df1, df2], ignore_index=True)
# Save combined data
combined_df.to_csv("chargingslot_requests_23456.csv", index=False)

C:\Users\sriz4651\AppData\Local\Temp\ipykernel_3324\2978913997.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([df1, df2], ignore_index=True)


In [44]:
# Constants
hover_power_watts = 159.01
battery_capacity_wh = 68.1
T_full_minutes = 80
factor = 0.2
tau = T_full_minutes * factor
alpha = 1.0  # heuristic-based weight factor

nodes_df = pd.read_csv('node_resource_count.csv')
drone_requests_df = pd.read_csv('chargingslot_requests_23456.csv')
requests_df = pd.read_csv("RequestsWithShortestPaths.csv")

# Convert ArrivalTime
start_date = datetime.strptime("2024-06-20 07:00", "%Y-%m-%d %H:%M")
drone_requests_df['ArrivalTime'] = drone_requests_df['ArrivalTime'].apply(
    lambda x: start_date + timedelta(minutes=int(x.split(":")[0]) * 60 + int(x.split(":")[1])) - timedelta(hours=7)
)

# Preprocess Request Data
requests_df['RequestID'] = requests_df['Request_ID']
requests_df['Battery_Consumption_Per_Segment (%)'] = requests_df['Battery_Consumption_Per_Segment (%)'].apply(ast.literal_eval)
requests_df['Node_IDs_Visited'] = requests_df['Node_IDs_Visited'].apply(ast.literal_eval)

# Battery lookup
battery_lookup = {}
for _, row in requests_df.iterrows():
    request_id = row['RequestID']
    nodes = row['Node_IDs_Visited']
    consumptions = row['Battery_Consumption_Per_Segment (%)']
    battery = 100
    battery_lookup[(request_id, nodes[0])] = battery
    for i in range(1, len(nodes)):
        battery -= consumptions[i - 1]
        battery_lookup[(request_id, nodes[i])] = battery

# Final SOC mapping
requests_df['Request_ID_Number'] = requests_df['Request_ID']
final_soc_map = requests_df.set_index('Request_ID_Number')['Final_SOC'].to_dict()

# Queues
queues_adjusted = {
    node: {"RechargingQueue": [], "WaitingQueue": [], "OverflowQueue": []}
    for node in drone_requests_df['NodeID'].unique()
}

default_recharging_queue_limit = 1
default_waiting_queue_limit = 2

events = []
status_records = []
hover_log = []
recharge_extension_log = []

current_time = start_date
end_time = max(drone_requests_df['ArrivalTime']) + timedelta(minutes=80)

while current_time <= end_time or any(
    queues_adjusted[node]['RechargingQueue'] or
    queues_adjusted[node]['WaitingQueue'] or
    queues_adjusted[node]['OverflowQueue']
    for node in queues_adjusted.keys()
):
    new_arrivals = drone_requests_df[drone_requests_df['ArrivalTime'] == current_time]
    for _, request in new_arrivals.iterrows():
        node_id = request['NodeID']
        arrival_time = request['ArrivalTime']
        recharging_time = request['RechargingTime']
        request_id = int(request['RequestID'])

        if recharging_time == 0:
            continue

        if node_id in nodes_df['NodeID'].values:
            recharging_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'RechargingQueueLimit'].values[0]
            waiting_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'WaitingQueueLimit'].values[0]
        else:
            recharging_queue_limit = default_recharging_queue_limit
            waiting_queue_limit = default_waiting_queue_limit

        if (len(queues_adjusted[node_id]['RechargingQueue']) < recharging_queue_limit and
            len(queues_adjusted[node_id]['WaitingQueue']) == 0 and
            len(queues_adjusted[node_id]['OverflowQueue']) == 0):
            queues_adjusted[node_id]['RechargingQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "RechargingQueue (Direct Arrival - All Queues Empty)"))
        elif len(queues_adjusted[node_id]['WaitingQueue']) < waiting_queue_limit:
            queues_adjusted[node_id]['WaitingQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "WaitingQueue (New Arrival)"))
        else:
            queues_adjusted[node_id]['OverflowQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "OverflowQueue (New Arrival)"))

    for node_id in queues_adjusted.keys():
        if node_id in nodes_df['NodeID'].values:
            recharging_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'RechargingQueueLimit'].values[0]
            waiting_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'WaitingQueueLimit'].values[0]
        else:
            recharging_queue_limit = default_recharging_queue_limit
            waiting_queue_limit = default_waiting_queue_limit

        # Remove drones done recharging
        for req in queues_adjusted[node_id]['RechargingQueue'][:]:
            request_id, start_time, recharge = req
            if current_time >= start_time + timedelta(minutes=recharge):
                queues_adjusted[node_id]['RechargingQueue'].remove(req)
                events.append((current_time, request_id, "Released from RechargingQueue"))

        # heuristic-based promotion to Recharging
        combined = []
        for request_id, arrival, recharge in queues_adjusted[node_id]['WaitingQueue']:
            wait_time = (current_time - arrival).total_seconds() / 60
            score = (wait_time ** 1.2) + alpha * recharge
            combined.append(("Waiting", request_id, arrival, recharge, recharge, score))

        for request_id, arrival, recharge in queues_adjusted[node_id]['OverflowQueue']:
            hover_time = (current_time - arrival).total_seconds() / 60
            battery_hover_pct = (hover_power_watts * hover_time / 60) / battery_capacity_wh * 100
            arrival_battery = battery_lookup.get((request_id, node_id), None)
            final_soc = final_soc_map.get(request_id, None)

            if arrival_battery is not None and final_soc is not None:
                total_pct = battery_hover_pct + arrival_battery
                if total_pct > 100:
                    Si = 0.11
                    Sf = final_soc
                else:
                    Si = 1 - (arrival_battery / 100)
                    Sf = Si + (battery_hover_pct / 100)
                if Sf >= 1 or Si >= Sf:
                    hover_recharge_time = 0
                else:
                    hover_recharge_time = -tau * math.log((1 - Sf) / (1 - Si))
                updated_recharge = recharge + hover_recharge_time
            else:
                updated_recharge = recharge

            wait_time = (current_time - arrival).total_seconds() / 60
            score = (wait_time ** 1.2) + alpha * updated_recharge
            combined.append(("Overflow", request_id, arrival, recharge, updated_recharge, score))

        combined.sort(key=lambda x: x[5])

        slots_available = recharging_queue_limit - len(queues_adjusted[node_id]['RechargingQueue'])
        promotions = combined[:slots_available]
        leftovers = combined[slots_available:]

        queues_adjusted[node_id]['WaitingQueue'] = []
        queues_adjusted[node_id]['OverflowQueue'] = []

        for origin, request_id, arrival, original_recharge, updated_recharge, score in promotions:
            if origin == "Overflow" and updated_recharge != original_recharge:
                queues_adjusted[node_id]['RechargingQueue'].append((request_id, current_time, updated_recharge))
                hover_time = (current_time - arrival).total_seconds() / 60
                battery_hover_pct = (hover_power_watts * hover_time / 60) / battery_capacity_wh * 100
                arrival_battery = battery_lookup.get((request_id, node_id), None)
                final_soc = final_soc_map.get(request_id, None)
                recharge_extension_log.append((request_id, node_id, original_recharge, updated_recharge, updated_recharge - original_recharge))
                hover_log.append((request_id, hover_time, battery_hover_pct, arrival_battery, final_soc, updated_recharge))
            else:
                queues_adjusted[node_id]['RechargingQueue'].append((request_id, current_time, original_recharge))

            events.append((current_time, request_id, f"Moved to RechargingQueue (WFS from {origin})"))

        # Promote remaining to waiting or overflow
        for origin, request_id, arrival, recharge, _, score in leftovers:
            if len(queues_adjusted[node_id]['WaitingQueue']) < waiting_queue_limit:
                queues_adjusted[node_id]['WaitingQueue'].append((request_id, arrival, recharge))
                events.append((current_time, request_id, f"Moved to WaitingQueue (Priority from {origin})"))
            else:
                queues_adjusted[node_id]['OverflowQueue'].append((request_id, arrival, recharge))
                events.append((current_time, request_id, f"Remains in OverflowQueue (After Priority Sort)"))

        recharging_status = [req[0] for req in queues_adjusted[node_id]['RechargingQueue']]
        waiting_status = [req[0] for req in queues_adjusted[node_id]['WaitingQueue']]
        overflow_status = [req[0] for req in queues_adjusted[node_id]['OverflowQueue']]
        status_records.append((current_time, node_id, recharging_status, waiting_status, overflow_status))

    current_time += timedelta(minutes=1)

# Save files
status_df = pd.DataFrame(status_records, columns=["Time", "NodeID", "RechargingQueue", "WaitingQueue", "OverflowQueue"])
hover_df = pd.DataFrame(hover_log, columns=["Request_ID", "HoveringTime_Minutes", "Battery_Hover%", "BatteryLevelOnArrival", "Final_SOC", "UpdatedRechargeTime"])
recharge_df = pd.DataFrame(recharge_extension_log, columns=["Request_ID", "Node_ID", "Original_Recharge", "Updated_Recharge", "Increase"])
status_df.sort_values(by=["NodeID", "Time"], inplace=True)
status_df.to_csv("node_status_6.csv", index=False)
hover_df.to_csv("hover_log2.csv", index=False)
recharge_df.to_csv("recharge_time_increase2.csv", index=False)

In [45]:
status_df = pd.read_csv('node_status_6.csv')
requests_df = pd.read_csv('chargingslot_requests_23456.csv')

# Convert ArrivalTime to datetime (current format HH:MM:SS)
requests_df['ArrivalTime'] = requests_df['ArrivalTime'].apply(
    lambda x: datetime.strptime(f"2024-06-20 {x}", "%Y-%m-%d %H:%M:%S")
    if isinstance(x, str) and len(x) == 8 else pd.to_datetime(x)
)

# Create lookup from (RequestID, NodeID) → ArrivalTime
arrival_lookup = requests_df.set_index(['RequestID', 'NodeID'])['ArrivalTime'].to_dict()

# Track first timestamp a drone appears in RechargingQueue
recharging_log = {}
for _, row in status_df.iterrows():
    time = pd.to_datetime(row['Time'])
    node_id = row['NodeID']
    try:
        recharging_q = eval(row['RechargingQueue'])  # Convert string to list
    except Exception:
        continue
    for request_id in recharging_q:
        key = (request_id, node_id)
        if key not in recharging_log:
            recharging_log[key] = time

# Prepare output with correct timestamps
waiting_records = []
for (request_id, node_id), recharge_start_time in recharging_log.items():
    key = (request_id, node_id)
    if key in arrival_lookup:
        arrival_time = arrival_lookup[key]
        waiting_time = (recharge_start_time - arrival_time).total_seconds() / 60
        waiting_records.append({
            'RequestID': request_id,
            'NodeID': node_id,
            'ArrivalTime': arrival_time,
            'RechargingStartTime': recharge_start_time,
            'WaitingTime_Minutes': round(waiting_time, 2)
        })

waiting_df = pd.DataFrame(waiting_records)
waiting_df.sort_values(by=['NodeID', 'ArrivalTime'], inplace=True)
waiting_df.to_csv('waiting_times_node23456.csv', index=False)

In [46]:
file_path = "RequestsWithShortestPaths.csv"
df = pd.read_csv(file_path)

# Check and create Updated_Trajectory_Time column
if 'Updated_Trajectory_Time23456' not in df.columns:
    df['Updated_Trajectory_Time23456'] = df['Trajectory_Time']
else:
    print(" 'Updated_Trajectory_Time23456' already exists, no changes made.")
df.to_csv("RequestsWithShortestPaths.csv", index=False)

In [47]:
requests_df = pd.read_csv("RequestsWithShortestPaths.csv")
waiting_df = pd.read_csv("waiting_times_node23456.csv")

# Convert Request_ID to numeric
requests_df['Request_ID'] = requests_df['Request_ID']
waiting_df['RequestID'] = waiting_df['RequestID'].astype(int)

# Parse trajectory time as dictionaries
requests_df['Updated_Trajectory_Time23456'] = requests_df['Trajectory_Time'].apply(ast.literal_eval)

# Function to apply waiting time delay from the specified node
def apply_waiting_time_delay(traj_dict, delay_node, wait_minutes):
    new_traj = {}
    delay_applied = False
    delay = timedelta(minutes=wait_minutes)
    for key, value in traj_dict.items():
        start_str, end_str = value.split('-')
        start = datetime.strptime(start_str, "%H:%M:%S.%f")
        end = datetime.strptime(end_str, "%H:%M:%S.%f")
        if key == delay_node:
            # Only extend the END time of the node
            end += delay
            delay_applied = True
        elif delay_applied:
            # Apply delay to all subsequent start and end times
            start += delay
            end += delay
        new_traj[key] = f"{start.strftime('%H:%M:%S.%f')[:-3]}-{end.strftime('%H:%M:%S.%f')[:-3]}"
    return new_traj

# Apply the delay to each request
for _, row in waiting_df.iterrows():
    req_id = row['RequestID']
    node = row['NodeID']
    wait_time = row['WaitingTime_Minutes']
    match_idx = requests_df[requests_df['Request_ID'] == req_id].index
    if not match_idx.empty:
        idx = match_idx[0]
        traj_dict = requests_df.at[idx, 'Updated_Trajectory_Time23456']
        updated_traj = apply_waiting_time_delay(traj_dict, node, wait_time)
        requests_df.at[idx, 'Updated_Trajectory_Time23456'] = updated_traj

# Convert dicts to string before saving
requests_df['Updated_Trajectory_Time23456'] = requests_df['Updated_Trajectory_Time23456'].apply(str)
requests_df.to_csv("RequestsWithShortestPaths.csv", index=False)

Processing the seventh node visited by the drones. The delay is propagated to the seventh node based on the impact of interferences at the previous node.

In [48]:
requests_df = pd.read_csv('RequestsWithShortestPaths.csv')

# Parse Trajectory_Time column
def parse_trajectory_times(row):
    try:
        return ast.literal_eval(row['Updated_Trajectory_Time23456'])
    except:
        return {}

requests_df['Parsed_Times'] = requests_df.apply(parse_trajectory_times, axis=1)

# Collect info for each drone and node
log_data = []
for idx, row in requests_df.iterrows():
    drone_id = row['Drone_ID']
    request_id = row['Request_ID']
    times_dict = row['Parsed_Times']

    for node, time_range in times_dict.items():
        if node.startswith("Node_"):
            try:
                start_str, end_str = time_range.split('-')
                start_time = datetime.strptime(start_str, "%H:%M:%S.%f")
                end_time = datetime.strptime(end_str, "%H:%M:%S.%f")
                recharge_time = round((end_time - start_time).total_seconds() / 60, 2)  # in minutes

                if recharge_time == 1.0:
                    recharge_time = 0

                log_data.append({
                    "Drone_ID": drone_id,
                    "RequestID": request_id,
                    "NodeID": node,
                    "ArrivalTime": start_time.strftime("%H:%M:%S"),
                    "RechargingTime": recharge_time
                })
            except Exception as e:
                print(f"Error processing node {node} for drone {drone_id}: {e}")

log_df = pd.DataFrame(log_data)
# Sort the DataFrame by NodeID and ArrivalTime
log_df.sort_values(by=["NodeID", "ArrivalTime"], inplace=True)
log_df.to_csv("chargingslot_requests_Full.csv", index=False)

In [49]:
df = pd.read_csv("chargingslot_requests_Full.csv") 
# Filter rows where RechargingTime > 0
filtered_df = df[df['RechargingTime'] > 0]
filtered_df.to_csv("chargingslot_requests_Full.csv", index=False)


In [50]:
paths_df = pd.read_csv("RequestsWithShortestPaths.csv")
charging_df = pd.read_csv("chargingslot_requests_Full.csv")

# Parse list from string
paths_df['Node_IDs_Visited'] = paths_df['Node_IDs_Visited'].apply(ast.literal_eval)

# Extract numeric request ID from string like 'Request_1212'
def extract_request_id(request_str):
    return int(str(request_str).split('_')[-1])

# Build mapping of (Drone_ID, RequestID) -> seventh visited node
seventh_node_map = {}
for _, row in paths_df.iterrows():
    drone_id = row['Drone_ID']
    request_id = extract_request_id(row['Request_ID'])
    visited_nodes = row['Node_IDs_Visited']
    if len(visited_nodes) >= 7:
        seventh_node_map[(drone_id, request_id)] = visited_nodes[6]

# Filter charging requests
filtered_charging_df = charging_df[
    charging_df.apply(
        lambda row: seventh_node_map.get((row['Drone_ID'], int(row['RequestID']))) == row['NodeID'],
        axis=1
    )
]

filtered_charging_df.to_csv("chargingslot_requests_7.csv", index=False)


In [51]:
df = pd.read_csv("chargingslot_requests_7.csv")

# Truncate seconds in the ArrivalTime column
def truncate_seconds(time_str):
    try:
        t = datetime.strptime(time_str, "%H:%M:%S")
        return t.replace(second=0).strftime("%H:%M:%S")
    except:
        return time_str  # fallback if parsing fails3
df['ArrivalTime'] = df['ArrivalTime'].apply(truncate_seconds)
df.to_csv("chargingslot_requests_7.csv", index=False)



In [52]:
df1 = pd.read_csv("chargingslot_requests_23456.csv")
df2 = pd.read_csv("chargingslot_requests_7.csv")
# Concatenate both files
combined_df = pd.concat([df1, df2], ignore_index=True)
combined_df.to_csv("chargingslot_requests_234567.csv", index=False)

C:\Users\sriz4651\AppData\Local\Temp\ipykernel_3324\3875920744.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([df1, df2], ignore_index=True)


In [53]:
# Constants
hover_power_watts = 159.01
battery_capacity_wh = 68.1
T_full_minutes = 80
factor = 0.2
tau = T_full_minutes * factor
alpha = 1.0  # heuristic-based weight factor

nodes_df = pd.read_csv('node_resource_count.csv')
drone_requests_df = pd.read_csv('chargingslot_requests_234567.csv')
requests_df = pd.read_csv("RequestsWithShortestPaths.csv")

# Convert ArrivalTime
start_date = datetime.strptime("2024-06-20 07:00", "%Y-%m-%d %H:%M")
drone_requests_df['ArrivalTime'] = drone_requests_df['ArrivalTime'].apply(
    lambda x: start_date + timedelta(minutes=int(x.split(":")[0]) * 60 + int(x.split(":")[1])) - timedelta(hours=7)
)

# Preprocess Request Data
requests_df['RequestID'] = requests_df['Request_ID']
requests_df['Battery_Consumption_Per_Segment (%)'] = requests_df['Battery_Consumption_Per_Segment (%)'].apply(ast.literal_eval)
requests_df['Node_IDs_Visited'] = requests_df['Node_IDs_Visited'].apply(ast.literal_eval)

# Battery lookup
battery_lookup = {}
for _, row in requests_df.iterrows():
    request_id = row['RequestID']
    nodes = row['Node_IDs_Visited']
    consumptions = row['Battery_Consumption_Per_Segment (%)']
    battery = 100
    battery_lookup[(request_id, nodes[0])] = battery
    for i in range(1, len(nodes)):
        battery -= consumptions[i - 1]
        battery_lookup[(request_id, nodes[i])] = battery

# Final SOC mapping
requests_df['Request_ID_Number'] = requests_df['Request_ID']
final_soc_map = requests_df.set_index('Request_ID_Number')['Final_SOC'].to_dict()

# Queues
queues_adjusted = {
    node: {"RechargingQueue": [], "WaitingQueue": [], "OverflowQueue": []}
    for node in drone_requests_df['NodeID'].unique()
}

default_recharging_queue_limit = 1
default_waiting_queue_limit = 2

events = []
status_records = []
hover_log = []
recharge_extension_log = []

current_time = start_date
end_time = max(drone_requests_df['ArrivalTime']) + timedelta(minutes=80)

while current_time <= end_time or any(
    queues_adjusted[node]['RechargingQueue'] or
    queues_adjusted[node]['WaitingQueue'] or
    queues_adjusted[node]['OverflowQueue']
    for node in queues_adjusted.keys()
):
    new_arrivals = drone_requests_df[drone_requests_df['ArrivalTime'] == current_time]
    for _, request in new_arrivals.iterrows():
        node_id = request['NodeID']
        arrival_time = request['ArrivalTime']
        recharging_time = request['RechargingTime']
        request_id = int(request['RequestID'])

        if recharging_time == 0:
            continue

        if node_id in nodes_df['NodeID'].values:
            recharging_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'RechargingQueueLimit'].values[0]
            waiting_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'WaitingQueueLimit'].values[0]
        else:
            recharging_queue_limit = default_recharging_queue_limit
            waiting_queue_limit = default_waiting_queue_limit

        if (len(queues_adjusted[node_id]['RechargingQueue']) < recharging_queue_limit and
            len(queues_adjusted[node_id]['WaitingQueue']) == 0 and
            len(queues_adjusted[node_id]['OverflowQueue']) == 0):
            queues_adjusted[node_id]['RechargingQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "RechargingQueue (Direct Arrival - All Queues Empty)"))
        elif len(queues_adjusted[node_id]['WaitingQueue']) < waiting_queue_limit:
            queues_adjusted[node_id]['WaitingQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "WaitingQueue (New Arrival)"))
        else:
            queues_adjusted[node_id]['OverflowQueue'].append((request_id, arrival_time, recharging_time))
            events.append((current_time, request_id, "OverflowQueue (New Arrival)"))

    for node_id in queues_adjusted.keys():
        if node_id in nodes_df['NodeID'].values:
            recharging_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'RechargingQueueLimit'].values[0]
            waiting_queue_limit = nodes_df.loc[nodes_df['NodeID'] == node_id, 'WaitingQueueLimit'].values[0]
        else:
            recharging_queue_limit = default_recharging_queue_limit
            waiting_queue_limit = default_waiting_queue_limit

        # Remove drones done recharging
        for req in queues_adjusted[node_id]['RechargingQueue'][:]:
            request_id, start_time, recharge = req
            if current_time >= start_time + timedelta(minutes=recharge):
                queues_adjusted[node_id]['RechargingQueue'].remove(req)
                events.append((current_time, request_id, "Released from RechargingQueue"))

        # heuristic-based promotion to Recharging
        combined = []
        for request_id, arrival, recharge in queues_adjusted[node_id]['WaitingQueue']:
            wait_time = (current_time - arrival).total_seconds() / 60
            score = (wait_time ** 1.2) + alpha * recharge
            combined.append(("Waiting", request_id, arrival, recharge, recharge, score))

        for request_id, arrival, recharge in queues_adjusted[node_id]['OverflowQueue']:
            hover_time = (current_time - arrival).total_seconds() / 60
            battery_hover_pct = (hover_power_watts * hover_time / 60) / battery_capacity_wh * 100
            arrival_battery = battery_lookup.get((request_id, node_id), None)
            final_soc = final_soc_map.get(request_id, None)

            if arrival_battery is not None and final_soc is not None:
                total_pct = battery_hover_pct + arrival_battery
                if total_pct > 100:
                    Si = 0.11
                    Sf = final_soc
                else:
                    Si = 1 - (arrival_battery / 100)
                    Sf = Si + (battery_hover_pct / 100)
                if Sf >= 1 or Si >= Sf:
                    hover_recharge_time = 0
                else:
                    hover_recharge_time = -tau * math.log((1 - Sf) / (1 - Si))
                updated_recharge = recharge + hover_recharge_time
            else:
                updated_recharge = recharge

            wait_time = (current_time - arrival).total_seconds() / 60
            score = (wait_time ** 1.2) + alpha * updated_recharge
            combined.append(("Overflow", request_id, arrival, recharge, updated_recharge, score))

        combined.sort(key=lambda x: x[5])

        slots_available = recharging_queue_limit - len(queues_adjusted[node_id]['RechargingQueue'])
        promotions = combined[:slots_available]
        leftovers = combined[slots_available:]

        queues_adjusted[node_id]['WaitingQueue'] = []
        queues_adjusted[node_id]['OverflowQueue'] = []

        for origin, request_id, arrival, original_recharge, updated_recharge, score in promotions:
            if origin == "Overflow" and updated_recharge != original_recharge:
                queues_adjusted[node_id]['RechargingQueue'].append((request_id, current_time, updated_recharge))
                hover_time = (current_time - arrival).total_seconds() / 60
                battery_hover_pct = (hover_power_watts * hover_time / 60) / battery_capacity_wh * 100
                arrival_battery = battery_lookup.get((request_id, node_id), None)
                final_soc = final_soc_map.get(request_id, None)
                recharge_extension_log.append((request_id, node_id, original_recharge, updated_recharge, updated_recharge - original_recharge))
                hover_log.append((request_id, hover_time, battery_hover_pct, arrival_battery, final_soc, updated_recharge))
            else:
                queues_adjusted[node_id]['RechargingQueue'].append((request_id, current_time, original_recharge))

            events.append((current_time, request_id, f"Moved to RechargingQueue (WFS from {origin})"))

        # Promote remaining to waiting or overflow
        for origin, request_id, arrival, recharge, _, score in leftovers:
            if len(queues_adjusted[node_id]['WaitingQueue']) < waiting_queue_limit:
                queues_adjusted[node_id]['WaitingQueue'].append((request_id, arrival, recharge))
                events.append((current_time, request_id, f"Moved to WaitingQueue (Priority from {origin})"))
            else:
                queues_adjusted[node_id]['OverflowQueue'].append((request_id, arrival, recharge))
                events.append((current_time, request_id, f"Remains in OverflowQueue (After Priority Sort)"))

        recharging_status = [req[0] for req in queues_adjusted[node_id]['RechargingQueue']]
        waiting_status = [req[0] for req in queues_adjusted[node_id]['WaitingQueue']]
        overflow_status = [req[0] for req in queues_adjusted[node_id]['OverflowQueue']]
        status_records.append((current_time, node_id, recharging_status, waiting_status, overflow_status))

    current_time += timedelta(minutes=1)

status_df = pd.DataFrame(status_records, columns=["Time", "NodeID", "RechargingQueue", "WaitingQueue", "OverflowQueue"])
hover_df = pd.DataFrame(hover_log, columns=["Request_ID", "HoveringTime_Minutes", "Battery_Hover%", "BatteryLevelOnArrival", "Final_SOC", "UpdatedRechargeTime"])
recharge_df = pd.DataFrame(recharge_extension_log, columns=["Request_ID", "Node_ID", "Original_Recharge", "Updated_Recharge", "Increase"])
status_df.sort_values(by=["NodeID", "Time"], inplace=True)
status_df.to_csv("node_status_7.csv", index=False)

In [54]:
status_df = pd.read_csv('node_status_7.csv')
requests_df = pd.read_csv('chargingslot_requests_234567.csv')

# Convert ArrivalTime to datetime (assuming format HH:MM:SS)
requests_df['ArrivalTime'] = requests_df['ArrivalTime'].apply(
    lambda x: datetime.strptime(f"2024-06-20 {x}", "%Y-%m-%d %H:%M:%S")
    if isinstance(x, str) and len(x) == 8 else pd.to_datetime(x)
)

# Create lookup from (RequestID, NodeID) → ArrivalTime
arrival_lookup = requests_df.set_index(['RequestID', 'NodeID'])['ArrivalTime'].to_dict()

# Track first timestamp a drone appears in RechargingQueue
recharging_log = {}
for _, row in status_df.iterrows():
    time = pd.to_datetime(row['Time'])
    node_id = row['NodeID']
    try:
        recharging_q = eval(row['RechargingQueue'])  # Convert string to list
    except Exception:
        continue
    for request_id in recharging_q:
        key = (request_id, node_id)
        if key not in recharging_log:
            recharging_log[key] = time

# Prepare output with correct timestamps
waiting_records = []
for (request_id, node_id), recharge_start_time in recharging_log.items():
    key = (request_id, node_id)
    if key in arrival_lookup:
        arrival_time = arrival_lookup[key]
        waiting_time = (recharge_start_time - arrival_time).total_seconds() / 60
        waiting_records.append({
            'RequestID': request_id,
            'NodeID': node_id,
            'ArrivalTime': arrival_time,
            'RechargingStartTime': recharge_start_time,
            'WaitingTime_Minutes': round(waiting_time, 2)
        })

waiting_df = pd.DataFrame(waiting_records)
waiting_df.sort_values(by=['NodeID', 'ArrivalTime'], inplace=True)
waiting_df.to_csv('waiting_times_node234567.csv', index=False)

In [55]:
file_path = "RequestsWithShortestPaths.csv"
df = pd.read_csv(file_path)

# Check and create Updated_Trajectory_Time column
if 'Updated_Trajectory_Time234567' not in df.columns:
    df['Updated_Trajectory_Time234567'] = df['Trajectory_Time']
else:
    print(" 'Updated_Trajectory_Time234567' already exists, no changes made.")
df.to_csv("RequestsWithShortestPaths.csv", index=False)

In [56]:
requests_df = pd.read_csv("RequestsWithShortestPaths.csv")
waiting_df = pd.read_csv("waiting_times_node234567.csv")

# Convert Request_ID to numeric
requests_df['Request_ID'] = requests_df['Request_ID'].astype(str).str.replace("Request_", "").astype(int)
waiting_df['RequestID'] = waiting_df['RequestID'].astype(int)

# Parse trajectory time as dictionaries
requests_df['Updated_Trajectory_Time234567'] = requests_df['Trajectory_Time'].apply(ast.literal_eval)

# Function to apply waiting time delay from the specified node
def apply_waiting_time_delay(traj_dict, delay_node, wait_minutes):
    new_traj = {}
    delay_applied = False
    delay = timedelta(minutes=wait_minutes)
    for key, value in traj_dict.items():
        start_str, end_str = value.split('-')
        start = datetime.strptime(start_str, "%H:%M:%S.%f")
        end = datetime.strptime(end_str, "%H:%M:%S.%f")
        if key == delay_node:
            # Only extend the END time of the node
            end += delay
            delay_applied = True
        elif delay_applied:
            # Apply delay to all subsequent start and end times
            start += delay
            end += delay
        new_traj[key] = f"{start.strftime('%H:%M:%S.%f')[:-3]}-{end.strftime('%H:%M:%S.%f')[:-3]}"
    return new_traj

# Apply the delay to each request
for _, row in waiting_df.iterrows():
    req_id = row['RequestID']
    node = row['NodeID']
    wait_time = row['WaitingTime_Minutes']
    match_idx = requests_df[requests_df['Request_ID'] == req_id].index
    if not match_idx.empty:
        idx = match_idx[0]
        traj_dict = requests_df.at[idx, 'Updated_Trajectory_Time234567']
        updated_traj = apply_waiting_time_delay(traj_dict, node, wait_time)
        requests_df.at[idx, 'Updated_Trajectory_Time234567'] = updated_traj

# Convert dicts to string before saving
requests_df['Updated_Trajectory_Time234567'] = requests_df['Updated_Trajectory_Time234567'].apply(str)
requests_df.to_csv("RequestsWithShortestPaths.csv", index=False)

In [57]:
df = pd.read_csv("RequestsWithShortestPaths.csv")

# Parse the dictionary column
df['Updated_Trajectory_Time234567'] = df['Updated_Trajectory_Time234567'].apply(ast.literal_eval)

# Function to calculate Actual delivery time due to the impact of inter-drone interference
def compute_duration_minutes(traj_dict):
    # Get all time ranges
    times = []
    for value in traj_dict.values():
        start_str, end_str = value.split('-')
        start = datetime.strptime(start_str, "%H:%M:%S.%f")
        end = datetime.strptime(end_str, "%H:%M:%S.%f")
        times.append((start, end))
    if not times:
        return 0
    # Get earliest start and latest end
    earliest_start = min(t[0] for t in times)
    latest_end = max(t[1] for t in times)
    duration_minutes = (latest_end - earliest_start).total_seconds() / 60
    return round(duration_minutes, 2)

# Apply the function to each row
df['AC_DT'] = df['Updated_Trajectory_Time234567'].apply(compute_duration_minutes)
df.to_csv("RequestsWithShortestPaths.csv", index=False)

In [58]:
df = pd.read_csv("RequestsWithShortestPaths.csv")

# Parse the dictionary column
df['Trajectory_Time'] = df['Trajectory_Time'].apply(ast.literal_eval)

# Function to calculate expected delivery time
def compute_duration_minutes(traj_dict):
    # Get all time ranges
    times = []
    for value in traj_dict.values():
        start_str, end_str = value.split('-')
        start = datetime.strptime(start_str, "%H:%M:%S.%f")
        end = datetime.strptime(end_str, "%H:%M:%S.%f")
        times.append((start, end))
    if not times:
        return 0
    # Get earliest start and latest end
    earliest_start = min(t[0] for t in times)
    latest_end = max(t[1] for t in times)
    duration_minutes = (latest_end - earliest_start).total_seconds() / 60
    return round(duration_minutes, 2)

# Apply the function to each row
df['EX_DT'] = df['Trajectory_Time'].apply(compute_duration_minutes)
df.to_csv("RequestsWithShortestPaths.csv", index=False)

In [59]:
df = pd.read_csv("RequestsWithShortestPaths.csv")

# Compute the raw difference
df['Additional_Waiting_Time'] = df['EX_DT'] - df['AC_DT']

# Apply logic: if negative, make positive; if positive or zero, make 0
df['Additional_Waiting_Time'] = df['Additional_Waiting_Time'].apply(
    lambda x: abs(x) if x < 0 else 0
)

# Save the updated DataFrame
df.to_csv("RequestsWithShortestPaths.csv", index=False)

In [60]:
# Constants
hover_power_watts = 159.01
battery_capacity_wh = 68.1

# Load status log
status_df = pd.read_csv('node_status_7.csv')
status_df['Time'] = pd.to_datetime(status_df['Time'])

# Convert string lists to actual lists
status_df['OverflowQueue'] = status_df['OverflowQueue'].apply(eval)
status_df['WaitingQueue'] = status_df['WaitingQueue'].apply(eval)
status_df['RechargingQueue'] = status_df['RechargingQueue'].apply(eval)

# Track drones in OverflowQueue and how long they stayed
drone_hover_times = {}

# Loop over time steps
for _, row in status_df.iterrows():
    timestamp = row['Time']
    node_id = row['NodeID']
    overflow = row['OverflowQueue']
    waiting = row['WaitingQueue']
    recharging = row['RechargingQueue']

    for drone_id in overflow:
        key = (drone_id, node_id)
        if key not in drone_hover_times:
            drone_hover_times[key] = {'start_time': timestamp, 'end_time': timestamp}
        else:
            drone_hover_times[key]['end_time'] = timestamp

    # Stop tracking drones that have moved out of overflow
    tracked_keys = list(drone_hover_times.keys())
    for key in tracked_keys:
        drone_id, nid = key
        if nid == node_id and drone_id not in overflow:
            # drone has exited overflow – no further tracking
            continue

# Final hover logs
records = []
for (drone_id, node_id), times in drone_hover_times.items():
    hover_minutes = (times['end_time'] - times['start_time']).total_seconds() / 60
    battery_used_wh = hover_minutes * (hover_power_watts / 60)
    battery_used_percent = (battery_used_wh / battery_capacity_wh) * 100

    records.append({
        'RequestID': drone_id,
        'NodeID': node_id,
        'StartTime': times['start_time'],
        'EndTime': times['end_time'],
        'HoveringMinutes': round(hover_minutes, 2),
        'Battery_Hover%': round(battery_used_percent, 2)
    })

hover_df = pd.DataFrame(records)
hover_df = hover_df.sort_values(by=['NodeID', 'StartTime'])
hover_df.to_csv("battery_consumed_overflow.csv", index=False)

In [61]:
hover_df = pd.read_csv('battery_consumed_overflow.csv')

# Clip values to a maximum of 80 as drones do emergency landings at this battery consumption
hover_df['Battery_Hover%'] = hover_df['Battery_Hover%'].clip(upper=80)
# Group by Request_ID and sum after clipping
hover_summary = hover_df.groupby('RequestID').agg({
    'Battery_Hover%': 'sum',
}).reset_index()

# Save the final hover summary CSV
hover_summary.to_csv('combined_hover_logUPD.csv', index=False)

In [62]:
recharge_df = pd.read_csv("combined_hover_logUPD.csv") 
requests_df = pd.read_csv("RequestsWithShortestPaths.csv") 

# Extract numeric part from Request_ID in requests file
requests_df['Request_ID'] = requests_df['Request_ID'].astype(str).str.extract(r'(\d+)', expand=False).astype(int)

# Map IncreasedRecharge based on Request_ID and fill missing with 0
requests_df['AC_BATTERY'] = requests_df['Request_ID'].map(
    recharge_df.set_index('RequestID')['Battery_Hover%']
).fillna(0)
requests_df.to_csv("RequestsWithShortestPaths.csv", index=False)


In [63]:
recharge_df = pd.read_csv("combined_hover_logUPD.csv")  
requests_df = pd.read_csv("RequestsWithShortestPaths.csv")

# Extract numeric part from Request_ID in requests file
requests_df['Request_ID'] = requests_df['Request_ID'].astype(str).str.extract(r'(\d+)', expand=False).astype(int)

# Map Battery_Hover% based on Request_ID
requests_df['Battery_Hover%'] = requests_df['Request_ID'].map(
    recharge_df.set_index('RequestID')['Battery_Hover%']
).fillna(0)

# Copy Total_Battery_Consumption (%) into EX_BATTERY
requests_df['EX_BATTERY'] = requests_df['Total_Battery_Consumption (%)']

# Compute AC_BATTERY = Battery_Hover% + EX_BATTERY
requests_df['AC_BATTERY'] = requests_df['Battery_Hover%'] +  requests_df['Total_Battery_Consumption (%)']
requests_df.to_csv("RequestsWithShortestPaths.csv", index=False)

In [64]:
df = pd.read_csv("RequestsWithShortestPaths.csv")

# Compute the raw difference
df['Additional_Battery'] = df['EX_BATTERY'] - df['AC_BATTERY']

# Apply logic: if negative, make positive; if positive or zero, make 0
df['Additional_Battery'] = df['Additional_Battery'].apply(
    lambda x: abs(x) if x < 0 else 0
)
df.to_csv("RequestsWithShortestPaths.csv", index=False)

In [65]:
df = pd.read_csv("RequestsWithShortestPaths.csv")

# Define Service_Status based on the two conditions
def determine_status(row):
    wait = row['Additional_Waiting_Time']
    battery = row['Additional_Battery']
    if wait > 0 and battery > 0:
        return "Not Serviced"
    elif wait > 0 or battery > 0:
        return "Partially Serviced"
    elif wait == 0 and battery == 0:
        return "Fully Serviced"
    else:
        return "Error Request"

# Apply the function to create Service_Status column
df['Service_Status'] = df.apply(determine_status, axis=1)
df.to_csv("RequestsWithShortestPaths.csv", index=False)

In [66]:
df = pd.read_csv("RequestsWithShortestPaths.csv")

# Count totals
total_requests = len(df)
fully_serviced = (df['Service_Status'] == 'Fully Serviced').sum()
not_serviced = (df['Service_Status'] == 'Not Serviced').sum()
Partial_serviced = (df['Service_Status'] == 'Partially Serviced').sum()
Error = (df['Service_Status'] == 'Error Request').sum()

# Print the results
print(f"Total Requests: {total_requests}")
print(f"Fully Serviced Requests: {fully_serviced}")
print(f"Partially Serviced Requests: {Partial_serviced}")
print(f"Not Serviced Requests: {not_serviced}")
print(f"Error Serviced Requests: {Error}")


Total Requests: 1320
Fully Serviced Requests: 975
Partially Serviced Requests: 184
Not Serviced Requests: 161
Error Serviced Requests: 0


In [67]:
df = pd.read_csv("RequestsWithShortestPaths.csv")

# Filter Partially Serviced requests
partial_df = df[df['Service_Status'] == 'Partially Serviced']
partial_avg_wait = partial_df['Additional_Waiting_Time'].mean()
partial_avg_battery = partial_df['Additional_Battery'].mean()

# Filter Not Serviced requests
not_serviced_df = df[df['Service_Status'] == 'Not Serviced']
not_serviced_avg_wait = not_serviced_df['Additional_Waiting_Time'].mean()
not_serviced_avg_battery = not_serviced_df['Additional_Battery'].mean()

# Print the results
print("🔸 Partially Serviced Requests:")
print(f"   Average Additional Wait Time: {partial_avg_wait:.2f} minutes")
print(f"   Average Additional Battery Use: {partial_avg_battery:.2f}%")

print("\n🔸 Not Serviced Requests:")
print(f"   Average Additional Wait Time: {not_serviced_avg_wait:.2f} minutes")
print(f"   Average Additional Battery Use: {not_serviced_avg_battery:.2f}%")


🔸 Partially Serviced Requests:
   Average Additional Wait Time: 8.38 minutes
   Average Additional Battery Use: 0.00%

🔸 Not Serviced Requests:
   Average Additional Wait Time: 95.54 minutes
   Average Additional Battery Use: 69.55%


In [68]:
approach_name = "Proposed Intervention" # Appraoch name
random_seed = 20  # random seed value for this run

df = pd.read_csv("RequestsWithShortestPaths.csv")

# Filter by service status
full_df = df[df['Service_Status'] == 'Fully Serviced']
partial_df = df[df['Service_Status'] == 'Partially Serviced']
not_serviced_df = df[df['Service_Status'] == 'Not Serviced']

# Compute counts
total_requests = len(df)
num_full = len(full_df)
num_partial = len(partial_df)
num_not_serviced = len(not_serviced_df)

# Compute averages
avg_full_wait = round(full_df['Additional_Waiting_Time'].mean(), 2)
avg_partial_wait = round(partial_df['Additional_Waiting_Time'].mean(), 2)
avg_not_wait = round(not_serviced_df['Additional_Waiting_Time'].mean(), 2)

avg_full_battery = round(full_df['Additional_Battery'].mean(), 2)
avg_partial_battery = round(partial_df['Additional_Battery'].mean(), 2)
avg_not_battery = round(not_serviced_df['Additional_Battery'].mean(), 2)

# Prepare summary row with Random_Seed
summary_data = {
    'Approach_Name': approach_name,
    'Total_Requests': total_requests,
    'Fully_Serviced': num_full,
    'Partially_Serviced': num_partial,
    'Not_Serviced': num_not_serviced,
    'Avg_Full_Wait_Time': avg_full_wait,
    'Avg_Partial_Wait_Time': avg_partial_wait,
    'Avg_Not_Wait_Time': avg_not_wait,
    'Avg_Full_Battery_Use': avg_full_battery,
    'Avg_Partial_Battery_Use': avg_partial_battery,
    'Avg_Not_Battery_Use': avg_not_battery,
    'Random_Seed': random_seed
}

summary_df = pd.DataFrame([summary_data])
output_file = 'ServiceSummary.csv'

# Append or create new file
if os.path.exists(output_file):
    summary_df.to_csv(output_file, mode='a', header=False, index=False)
else:
    summary_df.to_csv(output_file, index=False)

In [69]:
df = pd.read_csv('RequestsWithShortestPaths.csv')

# Define weights for computing QoS fulfillment score
alpha = 0.5
beta = 0.5

# Calculate QoS fulfillment score
def compute_qos(row):
    battery_dev = max(0, row['AC_BATTERY'] - row['EX_BATTERY']) / row['EX_BATTERY']
    time_dev = max(0, row['AC_DT'] - row['EX_DT']) / row['EX_DT']
    qos_score = 1 - (alpha * battery_dev + beta * time_dev)
    return round(qos_score, 4)

# Apply the function to each row
df['QoS_Fulfillment_Score'] = df.apply(compute_qos, axis=1)
# Clip final QoS score to ensure it's within 0 to 1 range
df['QoS_Fulfillment_Score'] = df['QoS_Fulfillment_Score'].clip(lower=0, upper=1)
df.to_csv('RequestsWithShortestPaths.csv', index=False)

In [70]:
code_end = datetime.now()
execution_time = (code_end - code_start).total_seconds()

In [71]:
approach_name = "Proposed Intervention"  # Approach name
random_seed = 20 # random seed for this run

df = pd.read_csv('RequestsWithShortestPaths.csv')
if 'QoS_Fulfillment_Score' not in df.columns:
    raise ValueError("The column 'QoS_Fulfillment_Score' is not present in the CSV file.")

# Compute QoS values
total_requests = len(df)
average_qos = round(df['QoS_Fulfillment_Score'].mean(), 4)

# Prepare summary row 
summary = pd.DataFrame([{
    'Approach_Name': approach_name,
    'Total_Requests': total_requests,
    'Avg_QoS_Fulfillment_Score': average_qos,
    'Random_Seed': random_seed 
}])

output_file = 'QoS_Summary.csv'
# Append if file exists, else create new with header
if os.path.exists(output_file):
    summary.to_csv(output_file, mode='a', header=False, index=False)
else:
    summary.to_csv(output_file, index=False)

In [72]:
approach_name = "Proposed Intervention"  # Approach name
random_seed = 20 # random seed value for this run

df_data = pd.read_csv('RequestsWithShortestPaths.csv')
total_requests = len(df_data)
df = pd.read_csv("node_status_7.csv")

# Convert WaitingQueue from string to list
df['WaitingQueue'] = df['WaitingQueue'].apply(ast.literal_eval)

# Compute congestion duration per node
congestion_duration = {}

for _, row in df.iterrows():
    node = row['NodeID']
    waiting_len = len(row['WaitingQueue'])
    if waiting_len >= 1:
        congestion_duration[node] = congestion_duration.get(node, 0) + 1

# Compute average congestion duration
total_congestion = sum(congestion_duration.values())
average_congestion_duration = round(total_congestion / 59, 2)

# Prepare summary row with Random_Seed column
summary_df = pd.DataFrame([{
    'Approach_Name': approach_name,
    'Number_of_Requests': total_requests,
    'Avg_Congestion_Duration': average_congestion_duration,
    'Random_Seed': random_seed 
}])

output_file = 'CongestionSummary.csv'
if os.path.exists(output_file):
    summary_df.to_csv(output_file, mode='a', header=False, index=False)
else:
    summary_df.to_csv(output_file, index=False)

In [73]:
approach_name = "Proposed Intervention"  # Approach name 
random_seed = 20  # random seed value for this run
output_file = "computation_time_log.csv"

# Compute execution time
execution_time = (code_end - code_start).total_seconds()

# Prepare summary row with Random_Seed column
log_entry = pd.DataFrame([{
    'Approach_Name': approach_name,
    'Number_of_Requests': total_requests,
    'Computation_Time_sec': execution_time,
    'Random_Seed': random_seed
}])

if os.path.exists(output_file):
    log_entry.to_csv(output_file, mode='a', index=False, header=False)
else:
    log_entry.to_csv(output_file, index=False)